In [1]:
# required package
# !pip install lda

In [2]:
import re
import json
import pandas as pd
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix

from lda_model import LDA_Model

In [3]:
emotion_df = pd.read_csv("emotion_dataset.csv")
emotion_df.head()

,text,label
0,“Worry is a down payment on a problem you may ...,optimism
1,My roommate: it's okay that we can't spell bec...,anger
2,No but that's so cute. Atsu was probably shy a...,joy
3,Rooneys fucking untouchable isn't he? Been fuc...,anger
4,it's pretty depressing when u hit pan on ur fa...,sadness


In [4]:
emotion_df["label"].unique()

array(['optimism', 'anger', 'joy', 'sadness'], dtype=object)

In [5]:
print("Dataframe length: ", len(emotion_df))
print("No. of NAN records: ", emotion_df["text"].isna().sum())

Dataframe length:  3257
No. of NAN records:  0


In [6]:
with open("seed_words.json") as file:
    seed_word = json.load(file)
print(seed_word["anger"][:5])

['outrageous', 'infuriating', 'ridiculous', 'absurd', 'exasperating']


Task 2:

Proposed preprocessing:

1. Keep alphabets only as other characters are not generating topic but might invoke the model to generate if abundant

2. Remove stop words to limit the search for topic related words (in this case, the abundance of stopwords might not affect the model as it is seeded already.)

3. To lowercase/uppercase as topics are not required here to be kept as mixed case

4. Lemmatization: the core form of words are needed rather than the participle forms to generate the topic

In [7]:
def clean_text(sentence: str) -> list:
    """cleaning text by extracting alphabets and
    then splitting into word list for each sentence

    Args:
        sentence (str): sentence string

    Returns:
        list: list of words.
    """
    pattern = re.compile(r"[A-Za-z]+")
    return re.findall(pattern=pattern, string=sentence)

def to_lowercase(word_list: list) -> list:
    """case changing of all contents in each list
    of words

    Args:
        word_list (list): list of words with alphabets only texts

    Returns:
        word_list (list): list of words with lowercase transformation
    """
    return [word.lower() for word in word_list]

def remove_stopwords(list_of_words: list) -> list:
    """removing stop words from list of words by matching
    English stop words and extracting those out 
    
    Args:
        list_of_words (list): list of words with stop words

    Returns:
        list: stop word free list of words
    """
    stopword_list = stopwords.words("english")

    return [word for word in list_of_words if word not in stopword_list]

def extract_word_lemma(word_list: list) -> list:
    """lemmatization of words list from word of list
    using NLTK WordNetLemmatizer class

    Args:
        word_list (list): list of string

    Returns:
        list: lemmatized list of strings
    """
    lemmatizer = WordNetLemmatizer()
    return [
        lemmatizer.lemmatize(word, tag[0].lower())
        if tag[0].lower() in ["n", "v", "a", "r", "s"]
        else word
        for word, tag in pos_tag(word_list)
    ]

In [8]:
#cleaning text
emotion_df["cleaned_text"] = emotion_df["text"].apply(lambda text: clean_text(text))
#removing stopwords
emotion_df["cleaned_text"] = emotion_df["cleaned_text"].apply(lambda word_list: remove_stopwords(word_list))
#to lowercase
emotion_df["cleaned_text"] = emotion_df["cleaned_text"].apply(lambda word_list: to_lowercase(word_list))
#lemmatize words
emotion_df["cleaned_text"] = emotion_df["cleaned_text"].apply(lambda word_list: extract_word_lemma(word_list))

Task 3

In [9]:
emotion_df["corpus"] = emotion_df["cleaned_text"].apply(lambda word_list: " ".join(word_list))
emotion_df.head()

,text,label,cleaned_text,corpus
0,“Worry is a down payment on a problem you may ...,optimism,"[worry, payment, problem, may, never, joyce, m...",worry payment problem may never joyce meyer mo...
1,My roommate: it's okay that we can't spell bec...,anger,"[my, roommate, okay, spell, autocorrect, terri...",my roommate okay spell autocorrect terrible fi...
2,No but that's so cute. Atsu was probably shy a...,joy,"[no, cute, atsu, probably, shy, photo, cherry,...",no cute atsu probably shy photo cherry help uwu
3,Rooneys fucking untouchable isn't he? Been fuc...,anger,"[rooneys, fuck, untouchable, be, fuck, dreadfu...",rooneys fuck untouchable be fuck dreadful depa...
4,it's pretty depressing when u hit pan on ur fa...,sadness,"[pretty, depress, u, hit, pan, ur, favourite, ...",pretty depress u hit pan ur favourite highlighter


In [10]:
K = 4
n_features = 10000
iterations = 5000

lda_models = []
for i in range(5):
    lda_models.append(LDA_Model(emotion_df["corpus"]))
    lda_models[i].documents_to_topic_model(
                        n_topics=K,
                        n_features=n_features,
                        n_iter=iterations,
                        original=True
                    )

Extracting tf features for LDA...
Extracting tf features for LDA...
Extracting tf features for LDA...
Extracting tf features for LDA...
Extracting tf features for LDA...


Task 4

In [11]:
seeded_lda = LDA_Model(emotion_df["corpus"])
seeded_lda.documents_to_topic_model(
    n_topics=K,
    n_features=n_features,
    n_iter=iterations,
    seed_words=seed_word
)

Extracting tf features for LDA...


In [14]:
seeded_lda.display_topics(1000)

Topic 0: user terrorism amp think say black outrage terror trump get people the anger not want you word bully war need fear this service serious make pakistan lie sad like terrible use back customer we if police kill tell really would white issue leave snap post that part give fight state something fuck happen mean protest absolutely shoot truth support world it no india insult don sadness die racism feel fact snapchat hold hillary yes they terrorist vote american store attack team shock destroy one human king stuff what medium true could stop within wanna another threaten never ask pic way america together provoke live even us horrible racist point cop offend man kik annoy please child where girl joke fuming mhchat be concern seem hilarity but party dm question imagine awful without speak safe despair send keep email problem sadly bore force sure create online charlotte agree rag etc adult care experience date month mind buy often biggest thing peace revenge usa election result rule n

In [15]:
topic0 = "user get like make go feel lose day one amp love see thing life anxiety im depression sad even people find need it work much really sadness happy good the today my little right hate look still time say do cry tell think start when take hard want smile stay shake week way night heart someone depress sleep be know something face glee ur year real family literally bit first nightmare you birthday bc pout everyone miss talk tomorrow job we friend cheer twitter new mourn how home lol angry since also either never always though well last nervous leave why end fear so change break post man sting die tear oh and around gloomy school worse worst bad help big sometimes phone long fall walk ask watch question eye thanks ever wait panic person mind yo she sound pretty gbbo lie laughter sober head forget revenge shy understand two wake car best playful fun rag laugh joyful mad seem full world tonight lot weary if half annoy hour dull this attack show alone fact us sparkle body sick hit excite hair hilarious horrid past dream gotta realize without hurt restless cant inside rage wtf gleeful cause whole stand music spend don mhchat piss outside stop age omg eat begin fucking now grim grow scar ready morning bed old keep cheery funny gonna with feeling bitter test lonely already child saw wish learn tip pizza relationship water remember name act ni away wear boy might front fuckin fan adult often depressing min later dog major hope pop offend val cat huff bar process yr mirth red broken mom picture photo dry guess everything that suppose cute second answer close extra damn different son glad finally high somebody bee machine kick kinda ago moment dear fiery death listen sister book pick member living dark who as three college stress summer suggest dude awesome tired awful awkward mood growl elate back drunk reality drink pain favorite empty accept dollar office sulk sure far manchester suffer matter option stuff difference exact exhaust plan mess stupid facebook holiday then instagram cuz kiss throw rapper truth irritation rejoice candice remind write sadly rest chuckle touch shall reverse can lay simple less dead pull read online focus class gym super sit grind song anyway cheerful waste okay monday momma cheerfully sweet future hand history city definitely fine forever experience john bus fill fright joy able win word rap wow reach secret website may hot form memory swear have fashion nigga blow short dread mother what bitch fake perfect mine shot friday beat cut carry men wonder haha absolutely interview situation irate saturday express pic loser easy yet wrong sing country ignore optimism video behind better ok thank deal there these debate officer decide deep th delight texan terrorism deserve despair destroy terror terrific terrible terrorist customer they complain control contact conflict concern con completely comedy cop come tiff coffee coach clown cloud cool could date dismal tho dangerous thought dad thousand threaten cunt cover ticket cross crime create crash crap discourage surprise divide enough episode etc store event stomach every everybody step steal statement exactly state star exhilarate exist expect st spirit spider speak source soul fail faith sort sorry soon some fast entire enjoy dm story term team taste target dont door draw dreadful clear drive drop drug support sunk suck due success dumb early earth style stuck struggle election else em email emotion straight till charlotte tire apparently assume within woman article arm argument app attempt anything anyone anymore worried worry another at average believe bb wall wanna become beautiful war weak ball avoid weekend whatever where white awe will worth animosity animate address yes aggravate yesterday afternoon afraid affliction add animal actually action young account your absolute agree ahs air alarm album all almost yeah yall ya amaze america american wrath an anger would vote voice together try truly catch trump case care card turn centre cannot tv cancel tweet campaign cam celtic challenge bet treat top chirp towards track train trash check chance charlotteprotest snapchat charge character channel true call buy but blind boil usa blue bloody blood block blame business black use useless very bill biggest update bore box unhappy unga breezy bright bring broadcast ugh brother brown btw bully burn burst typical social fire snap lively longer ppl president price lmao pride live lord probably problem promise lip line protest power positive provoke police many please plus pm point madden policy poor ly lunch luck lt low political light pun mary kid race king racism kind kill kik racist rabid key radio just jump joyous rape kitchen lack levity leadership level let punch punny least pure lead lady push put queue quite late quote mark match smoke or nice opportunity next news optimist network negative opinion order near nd nation others otherwise nicole open muslim only offer offense on obviously obama nut nothing no note not north normal nope non must musically mate place minute pessimist peter milk michelle pine play mistake meet medium mean me maybe player peace pay outrage most murder mufc pak movie move pakistan parent paul mope park month money part party joke join rather series gloom send sense sentence give girl george goal serious garbage game seriously fury furious self god set scare sam guy gun gt grudge save group season scary scene grieve great scorn scrum service fuming rd sink floor flight fix single finish finger sky follow final film smell fight smh felt folk food fume free shit shock shoot fuck frown shut freak foot forward side sign force for football safe rush run report incense in important imagine replace ill idiot indian idea id resent human resentment respect india info hang iphone jesus jazz issue islam reason irritate involve infuriate intimidate internet insult instead record relentless house respond horror he hell hearty room heard hear rooney haunt horrific hath round hat ruin rule happen rojo hey heyday hi rock road hilarity hillary rn rip hold ring revolt honestly return horrible result zero"
topic0 = topic0.split()

topic1 = "user people like say terrorism fuck really amp black outrage terror trump way make need want take get think try offend use war you pakistan anger the let police bully look word fury it if hell go not kill die woman give fear nothing insult problem serious right would no talk he this they and awful irritate someone pay ask white joke man world we country but shit sad fight attack state stop well something yes protest seriously concern all support fume boil shoot blood racism don revenge never india why vote hillary speak destroy yeah can hope terrorist threaten money sure human king keep afraid thing reason what mean know men gun better drive sort phone provoke cop that racist america tweet towards away point wait pretty decide date rabid break could resent weak hand even might soul rule miss call bitch come future class truth ppl some shot party levity care suppose maybe wanna on rag child cheer etc help force crime bright charlotte ugh without fuming these power be hold fact instead line sober president train side scene muslim centre sense islam mistake usa join unga peace despair whole day only race election must obviously respect agree nation jump mine resentment fake in brother bad chirp indian charlotteprotest many opinion lead every officer or pak save affliction gloom rush kinda conflict non order obama situation promise stuff shut bloody address exactly honestly deserve info for murder blind ignore policy animosity revolt stupid target offense offer spider spirit parent em lunch outside idiot id statement big rape dangerous american biggest leave guy average otherwise typical form scorn fiery wrath hath treat add sam smoke imagine argument carry express sentence enjoy term happen self seem due where level hot poor horrific moment freak divide shock bore whatever just ruin back quote shy pull growl exist pride put matter online cut intimidate depress very dont case nightmare dude brown nigga city hey front plan mother light report apparently check see cheerfully business guess act thought fashion emotion buy kind pick laugh accept secret experience lt finger weekend fire football waste wtf gym im broken job kid rage rapper grudge good cheerful time feeling gloomy hate felt gleeful forward glee already afternoon fill hat glad age film girl gt feel ago goal head fall grim grieve family absolute grind great gotta group fan absolutely grow far have haunt gonna god fast favorite adult aggravate actually george folk first fuckin fix album flight floor frown focus half follow forget food foot fright friend friday hang free account forever fucking hair alarm haha also ahs gbbo final garbage finally find hard fine game furious funny fun action finish air full alone happy almost bb faith cheery character channel change chance arm challenge celtic cause catch cat card car cant cannot candice cancel campaign charge chuckle around clear anyone cool control contact anything anyway con completely complain comedy college coffee coach clown cloud app close cam article fail bitter birthday bill awe awesome awkward bet best believe behind begin bee bed ball become beautiful beat bc bit blame bus block burst burn as assume btw at broadcast bring breezy attempt boy box book avoid body blue blow cover crap crash end email else elate either eat easy earth early dumb dull dry drunk an drug drop bar drink empty enough create entire facebook face eye extra expect exhilarate exhaust excite exact always everything everyone everybody amaze ever event episode dream angry animal dreadful anxiety debate death dear deal dead heard dark damn dad cuz cute customer cunt cry anymore cross answer deep another dm dread draw door animate dollar dog do dismal definitely discourage different difference annoy depression depressing delight hear zero heart sister so snapchat snap smile smh smell sleep sky sit sink somebody single sing since simple sign sick show short she social sometimes shake start straight story store stomach sting still step steal stay star son stand st spend sparkle source sound sorry soon song shall set struggle relentless reverse return result restless rest respond replace remind remember relationship rip rejoice red record realize reality real ready read reach ring rn service saw series send second season scrum school scary scare scar saturday road safe sadness sadly run round rooney room rojo rock stress stuck hearty wake when week website weary wear water watch wall walk voice will video val useless us ur update unhappy understand two who win tv write your young yo yet yesterday year yall ya wrong wow wish worth worst worse worry worried work wonder within with twitter turn style sweet th texan test terrific terrible tell tear team taste swear thanks surprise super sunk summer sulk suggest suffer suck success thank then truly tired true trash track touch top tonight tomorrow together today tire there tip till tiff ticket throw three thousand though tho rd rather rap little yr longer long lonely lol lmao living lively live literally lose listen lip life lie less least learn leadership lay lord loser later mary mess memory member meet medium me may mate match mark lot manchester major madden mad machine ly luck low love laughter late radio horrid important ill idea hurt huff how house hour horror horrible infuriate home holiday hit history hilarity hilarious high hi heyday incense inside last joy lady lack kitchen kiss kik kick key joyous joyful john instagram jesus jazz issue irritation irate iphone involve interview internet mhchat michelle milk paul piss pine picture pic photo peter pessimist person perfect past place part park panic pain others option optimist optimism opportunity pizza play min probably quite queue question push pure punny punch pun process price player pout post positive pop political pm plus please playful open one omg most name my musically music mufc much movie move mourn morning old mope mood month monday momma mom mirth minute mind nd near negative nervous okay ok oh often office nut now note north normal nope night nicole nice ni next news new network able"
topic1 = topic1.split()

topic2 = "user get go amp the back like would blue know time love new it start horror look work come turn play girl every fuck good today try year always great this next could hilarious furious say old sorry watch please alarm angry sad happen wish run just think thank kid night shit leave oh burn guy so house move movie player follow us wrong day expect you free late last lol see show one there tonight much many man that tv dreadful nightmare wanna home if rage do month buy call haunt awe bully fire first meet as idea game hilarity dark terrible ticket smile frown ok be wait bring another panic become breezy flight sit away horrible tweet face lip listen animate open absolutely music lmao god match sing relentless remember optimism room nd story serious fume delight together end discourage hear dream week pm put no news th video place grim gbbo top keep boy still gonna felt rejoice hour ball afternoon soon rather episode madden door pessimist crap social film half take safe food write funny ya anything mind kind book dont box foot mean create season scary hey rock step two morning friend early finish pure infuriate long lot depress interview attempt wonder clown fast stop style app your milk voice absolute negative beautiful lady rest nervous make maybe may mate aggravate hope wow rooney air head post lord find block coffee st trash dread deep debate fan swear track dad taste thanks crash forward friday set shock clear shake holiday in short cloud can not mark longer heyday believe stuck star an joyous radio cam dumb series hi steal easy age happy we hell mufc till tire tiff tomorrow minute terrific catch account tho channel animal nice store punch opportunity thousand nut poor campaign celtic loser political album normal surprise ahs park comedy mary cool pine cunt close check floor beat btw round where ill sky fix fill rojo website rd smell jazz email wall folk scrum have twitter scare far part bus yall weekend finger hit kill source win luck rip contact football brown depressing order incense exhilarate punny city pun smh parent saw optimist yeah single eat memory song little enough business imagine cut sometimes finally instead seriously fright anymore who john within let life report yesterday weary super then avoid apparently but fail divide medium due hang all definitely able others trump rap useless pride shoot sure kiss amaze agree promise he me tear anyway date sweet suppose deal dear talk death dangerous decide damn dead often support target dog suck dm suffer dismal suggest different difference die destroy despair deserve depression sulk summer sunk customer cuz cute completely complain texan these college they coach thing though thought threaten class three chuckle chirp con concern conflict term team don cry cross crime tell terror test cover country terrorism cop terrorist control dollar draw success faith fact facebook sound eye extra express experience sparkle exist speak exhaust excite exactly exact everything soul fake everybody fall final some somebody fight fiery someone feeling feel fear favorite something fashion son sort family everyone spend stupid stay sting stomach dull straight dude dry drunk drug drop drive drink stress struggle stuff cheery earth statement ever either event even etc spider entire spirit enjoy stand empty emotion state em else election elate child charlotteprotest cheerfully animosity assume ask what article around arm argument whatever when white anyone whole anxiety answer why at attack well bad want war bc bb bar waste water average awkward awful awesome way weak wear annoy will cheerful with wrath wtf yes afraid affliction adult address add actually action act yet accept yo young ago worth worst america without anger and woman word american world worse also already alone almost worried worry walk wake bed bee card car cant cannot candice cancel towards train treat true truly truth typical burst ugh care carry case tip cheer throw snapchat charlotte charge character change cat chance challenge centre tired cause touch understand unga unhappy biggest black bitter bitch bit birthday bill big blind better bet best vote behind begin blame very brother usa broken broadcast update bright ur break use blood bore boil body val blow bloody sober sleep fine lt positive pout power lose ppl yr president pretty lonely price living lively live probably literally low pop snap lunch picture matter piss pizza plan playful plus manchester point police major policy mad machine ly problem process line light lack quite kitchen king kinda quote rabid kik race kick key racism racist jump rag queue question push pull protest lie levity level provoke less least later learn leadership lead lay laughter laugh pick pic photo must nigga nicole option ni or otherwise never network outrage outside need near pain nation name opinion only online okay office offer offense offend obviously obama now non nothing note omg north on nope my muslim phone musically mistake miss mirth people mine perfect min person might michelle mhchat mess men peter member mom moment momma mourn pak murder pakistan party past paul mother monday most pay mope mood peace money joyful joy joke seem save scar scene goal school gloomy gloom gleeful glee glad give scorn second george secret gotta saturday grieve gun hand sadly hair haha gym sadness guess sam gt grudge growl grow group grind garbage future rule fury forever force for shy sick side sign simple focus since sink sister situation officer smoke forget form shut fucking self send fun fuming sense full fuckin freak sentence service front shall she shot rush ruin join human ready info indian india real reality important realize im really ignore reason idiot record id inside instagram read irritation job jesus rape rapper issue islam irritate insult irate iphone involve intimidate reach internet hurt huff hard how return revenge reverse help revolt hearty heart heard right ring rn road hath hate hat high result restless replace red relationship hot remind horrific horrid resent hillary honestly resentment respect hold respond history zero"
topic2 = topic2.split()

topic3 = "user get live watch think worry one know want amaze fear let never horrible even ever anger team life give shock terrible snap service broadcast musically time game ly see the angry unhappy still awful you play what well don customer yet lively someone worst tell bad use actually would anyone my bitter gt guy back afraid rage hold issue part people fuck us god sadly depression quote call best sink bb discourage drop when send point else that take but just least anything put snapchat burst minute deal at another sadness how come win person enough true chance stop probably grudge season lie wrath he lol already they may nothing read news despair straight key feel paul kik please cover almost believe also medium mean struggle dm suck okay sunk horrific be pic last everything within cross road heard she change hear bet success offense side record no care month nicole fuming wow faith level cause involve history maybe nope sign show peter contact final result plus lt fume control zero american texan store truly michelle hand con complain hang happen entire coach lack everybody assume forever year hearty blame action intimidate dismal network group test big will replace respond drug goal return bill low draw price optimism house challenge arm smh card around help experience internet near anymore charge now must cancel email stomach push kitchen thought honestly pine haha worried earth positive garbage important george note cannot rn north completely fail queue mope since exhilarate bore iphone character jesus update have event article thanks hat your name young most quite leadership worth ring death behind biggest st grieve moment avoid scare irate optimist blow perfect useless me business literally heart saturday better top rip pay high emotion damn freak tire voice others very focus reach kind self talk sweet ruin yesterday case light ago dear whatever shy incense single exist less rojo fine fuckin joy dull address leave forget guess everyone horrid lmao hey force for form absolutely hell accept account forward head act annoy haunt hath football hillary heyday foot finally hour find hot horror finger answer hope finish home fire holiday first absolute hit fix flight floor folk follow friday hilarity hilarious food hi free grim friend gloomy affliction girl alone glad all afternoon growl glee age gleeful gloom go gbbo aggravate album gonna grow good alarm gotta agree air ahs great always gun hate full fright hard add happy grind front frown animosity fucking half hair animate america animal and an amp gym fun funny furious fury future adult btw excite anxiety crap country could black cop cool conflict concern comedy blind block college coffee blood bloody clown cloud close bitch crash depress create deep decide debate dead day date dark dangerous dad cuz cute cut cunt cry birthday bit crime clear class city chuckle bright catch cat carry bring car cant candice can campaign cam broken buy brother bus brown burn breezy celtic centre boil chirp blue child body cheery cheerfully cheerful cheer break check book charlotteprotest charlotte box channel boy definitely depressing film as eye extra ask express expect exhaust bully exactly exact every attack etc attempt episode average away enjoy face facebook deserve fact fill fight fiery felt feeling anyway app apparently argument favorite fast fashion far fan family fall fake awe end empty awesome dream dreadful dread door dont dollar dog do become bed bee divide begin different difference die destroy beautiful drink drive bar em awkward election elate either eat ball easy beat bc early dumb due dude dry drunk delight look huff smile song son sometimes something somebody some social sober so smoke smell shoot sleep sky situation sit sister sing simple sick shut shot soon sorry sort soul stupid stuff stuck stress story sting step steal stay statement state start star stand spirit spider spend speak sparkle source sound short shit suffer respect round rooney room rock right revolt reverse revenge restless rest resentment shall resent report remind remember relentless relationship rejoice red reason really rule run rush sad shake set seriously serious series sentence sense seem secret second scrum scorn school scene scary scar say saw save sam safe style suggest reality walk weary wear weak we way water waste war wanna wall wake twitter wait vote video val usa ur unga understand ugh typical website week weekend where yo yes yeah yall ya wtf wrong write worse world work word wonder woman without with wish why whole who white two tweet sulk terrific this thing these there then thank th terrorist terrorism terror term tv tear taste target swear surprise sure suppose support super summer tho though thousand threaten turn try truth trump treat trash train track towards touch tonight tomorrow together today tired tip till tiff ticket throw three realize real human luck mark many manchester man make major madden mad machine lunch love lip lot loser lose lord yr longer long lonely living little mary match mate matter mother morning mood money monday momma mom mistake miss mirth mine mind min milk might mhchat mess men memory member meet listen line move indian islam irritation irritate interview insult instead instagram inside infuriate info india like in imagine im ill ignore if idiot idea id hurt it jazz job john levity learn lead lay laughter laugh later late lady kiss king kinda kill kid kick keep jump joyous joyful joke join mourn movie ready playful ppl power pout post pop poor political policy police pm player party plan place pizza piss picture pick photo phone pessimist peace president pretty pride problem rd rather rapper rape rap rag radio racist racism race rabid question pure punny punch pun pull provoke protest promise process past park much ni obviously obama nut not normal non nightmare night nigga nice next parent new nervous negative need nd nation muslim music murder mufc offend offer office officer panic pakistan pak pain outside outrage otherwise order or option opportunity opinion open only online on omg old ok oh often able"
topic3 = topic3.split()

In [16]:
len(topic2)

1000

In [17]:
list(seed_word.keys())

['anger', 'sadness', 'joy', 'optimism']

In [18]:
topic_words = [topic0, topic1, topic2, topic3]
max_proba = 0
for i, topic in enumerate(topic_words):
    for key, value in seed_word.items():
        common_words = set(topic) & set(value)
        common_words_count = len(common_words)
        percentage = (common_words_count / len(value)) * 100
        print(f"[Info:] For Topic {i}, Percentage of words in emotion: {key.upper()} = {percentage: .2f}%")
        print(f"Common words are: {common_words}")
    print("\n=====================\n")

[Info:] For Topic 0, Percentage of words in emotion: ANGER =  17.07%
Common words are: {'angry', 'mad', 'irate', 'bitter', 'outrage', 'cross', 'furious'}
[Info:] For Topic 0, Percentage of words in emotion: SADNESS =  11.11%
Common words are: {'unhappy', 'sad', 'blue', 'dismal', 'gloomy'}
[Info:] For Topic 0, Percentage of words in emotion: JOY =  13.33%
Common words are: {'positive', 'joyful', 'gleeful', 'happy', 'cheery', 'cheerful'}
[Info:] For Topic 0, Percentage of words in emotion: OPTIMISM =  8.89%
Common words are: {'joyful', 'cheerful', 'bright', 'positive'}


[Info:] For Topic 1, Percentage of words in emotion: ANGER =  17.07%
Common words are: {'angry', 'mad', 'irate', 'bitter', 'outrage', 'cross', 'furious'}
[Info:] For Topic 1, Percentage of words in emotion: SADNESS =  11.11%
Common words are: {'unhappy', 'sad', 'blue', 'dismal', 'gloomy'}
[Info:] For Topic 1, Percentage of words in emotion: JOY =  13.33%
Common words are: {'positive', 'joyful', 'gleeful', 'happy', 'cheer

I noticed almost equal distribution of each topics. So, here ANGER is dominant
whereas JOY, SADNESS, and lastly, OPTIMISM is taking place.
I analyzed 1000 words and then calculated their probability of appearance for each topic

Task 5

In [19]:
for i, model in enumerate(lda_models):
    print(f"Topics of model {i+1}:")
    model.display_topics(1000)
    print("\n================\n")

Topics of model 1:
Topic 0: user terrorism amp think say black outrage terror trump get people the anger not want you word bully war need fear this service serious make pakistan lie sad like terrible use back customer we if police kill tell really would white issue leave snap post that part give fight state something fuck happen mean protest absolutely shoot truth support world it no india insult don sadness die racism feel fact snapchat hold hillary yes they terrorist vote american store attack team shock destroy one human king stuff what medium true could stop within wanna another threaten never ask pic way america together provoke live even us horrible racist point cop offend man kik annoy please child where girl joke fuming mhchat be concern seem hilarity but party dm question imagine awful without speak safe despair send keep email problem sadly bore force sure create online charlotte agree rag etc adult care experience date month mind buy often biggest thing peace revenge usa ele

In [20]:
unseeded_model_1 = [
    "user get like make go feel lose day one amp love see thing life anxiety im depression sad even people find need it work much really sadness happy good the today my little right hate look still time say do cry tell think start when take hard want smile stay shake week way night heart someone depress sleep be know something face glee ur year real family literally bit first nightmare you birthday bc pout everyone miss talk tomorrow job we friend cheer twitter new mourn how home lol angry since also either never always though well last nervous leave why end fear so change break post man sting die tear oh and around gloomy school worse worst bad help big sometimes phone long fall walk ask watch question eye thanks ever wait panic person mind yo she sound pretty gbbo lie laughter sober head forget revenge shy understand two wake car best playful fun rag laugh joyful mad seem full world tonight lot weary if half annoy hour dull this attack show alone fact us sparkle body sick hit excite hair hilarious horrid past dream gotta realize without hurt restless cant inside rage wtf gleeful cause whole stand music spend don mhchat piss outside stop age omg eat begin fucking now grim grow scar ready morning bed old keep cheery funny gonna with feeling bitter test lonely already child saw wish learn tip pizza relationship water remember name act ni away wear boy might front fuckin fan adult often depressing min later dog major hope pop offend val cat huff bar process yr mirth red broken mom picture photo dry guess everything that suppose cute second answer close extra damn different son glad finally high somebody bee machine kick kinda ago moment dear fiery death listen sister book pick member living dark who as three college stress summer suggest dude awesome tired awful awkward mood growl elate back drunk reality drink pain favorite empty accept dollar office sulk sure far manchester suffer matter option stuff difference exact exhaust plan mess stupid facebook holiday then instagram cuz kiss throw rapper truth irritation rejoice candice remind write sadly rest chuckle touch shall reverse can lay simple less dead pull read online focus class gym super sit grind song anyway cheerful waste okay monday momma cheerfully sweet future hand history city definitely fine forever experience john bus fill fright joy able win word rap wow reach secret website may hot form memory swear have fashion nigga blow short dread mother what bitch fake perfect mine shot friday beat cut carry men wonder haha absolutely interview situation irate saturday express pic loser easy yet wrong sing country ignore optimism video behind better ok thank deal there these debate officer decide deep th delight texan terrorism deserve despair destroy terror terrific terrible terrorist customer they complain control contact conflict concern con completely comedy cop come tiff coffee coach clown cloud cool could date dismal tho dangerous thought dad thousand threaten cunt cover ticket cross crime create crash crap discourage surprise divide enough episode etc store event stomach every everybody step steal statement exactly state star exhilarate exist expect st spirit spider speak source soul fail faith sort sorry soon some fast entire enjoy dm story term team taste target dont door draw dreadful clear drive drop drug support sunk suck due success dumb early earth style stuck struggle election else em email emotion straight till charlotte tire apparently assume within woman article arm argument app attempt anything anyone anymore worried worry another at average believe bb wall wanna become beautiful war weak ball avoid weekend whatever where white awe will worth animosity animate address yes aggravate yesterday afternoon afraid affliction add animal actually action young account your absolute agree ahs air alarm album all almost yeah yall ya amaze america american wrath an anger would vote voice together try truly catch trump case care card turn centre cannot tv cancel tweet campaign cam celtic challenge bet treat top chirp towards track train trash check chance charlotteprotest snapchat charge character channel true call buy but blind boil usa blue bloody blood block blame business black use useless very bill biggest update bore box unhappy unga breezy bright bring broadcast ugh brother brown btw bully burn burst typical social fire snap lively longer ppl president price lmao pride live lord probably problem promise lip line protest power positive provoke police many please plus pm point madden policy poor ly lunch luck lt low political light pun mary kid race king racism kind kill kik racist rabid key radio just jump joyous rape kitchen lack levity leadership level let punch punny least pure lead lady push put queue quite late quote mark match smoke or nice opportunity next news optimist network negative opinion order near nd nation others otherwise nicole open muslim only offer offense on obviously obama nut nothing no note not north normal nope non must musically mate place minute pessimist peter milk michelle pine play mistake meet medium mean me maybe player peace pay outrage most murder mufc pak movie move pakistan parent paul mope park month money part party joke join rather series gloom send sense sentence give girl george goal serious garbage game seriously fury furious self god set scare sam guy gun gt grudge save group season scary scene grieve great scorn scrum service fuming rd sink floor flight fix single finish finger sky follow final film smell fight smh felt folk food fume free shit shock shoot fuck frown shut freak foot forward side sign force for football safe rush run report incense in important imagine replace ill idiot indian idea id resent human resentment respect india info hang iphone jesus jazz issue islam reason irritate involve infuriate intimidate internet insult instead record relentless house respond horror he hell hearty room heard hear rooney haunt horrific hath round hat ruin rule happen rojo hey heyday hi rock road hilarity hillary rn rip hold ring revolt honestly return horrible result zero",
    "user people like say terrorism fuck really amp black outrage terror trump way make need want take get think try offend use war you pakistan anger the let police bully look word fury it if hell go not kill die woman give fear nothing insult problem serious right would no talk he this they and awful irritate someone pay ask white joke man world we country but shit sad fight attack state stop well something yes protest seriously concern all support fume boil shoot blood racism don revenge never india why vote hillary speak destroy yeah can hope terrorist threaten money sure human king keep afraid thing reason what mean know men gun better drive sort phone provoke cop that racist america tweet towards away point wait pretty decide date rabid break could resent weak hand even might soul rule miss call bitch come future class truth ppl some shot party levity care suppose maybe wanna on rag child cheer etc help force crime bright charlotte ugh without fuming these power be hold fact instead line sober president train side scene muslim centre sense islam mistake usa join unga peace despair whole day only race election must obviously respect agree nation jump mine resentment fake in brother bad chirp indian charlotteprotest many opinion lead every officer or pak save affliction gloom rush kinda conflict non order obama situation promise stuff shut bloody address exactly honestly deserve info for murder blind ignore policy animosity revolt stupid target offense offer spider spirit parent em lunch outside idiot id statement big rape dangerous american biggest leave guy average otherwise typical form scorn fiery wrath hath treat add sam smoke imagine argument carry express sentence enjoy term happen self seem due where level hot poor horrific moment freak divide shock bore whatever just ruin back quote shy pull growl exist pride put matter online cut intimidate depress very dont case nightmare dude brown nigga city hey front plan mother light report apparently check see cheerfully business guess act thought fashion emotion buy kind pick laugh accept secret experience lt finger weekend fire football waste wtf gym im broken job kid rage rapper grudge good cheerful time feeling gloomy hate felt gleeful forward glee already afternoon fill hat glad age film girl gt feel ago goal head fall grim grieve family absolute grind great gotta group fan absolutely grow far have haunt gonna god fast favorite adult aggravate actually george folk first fuckin fix album flight floor frown focus half follow forget food foot fright friend friday hang free account forever fucking hair alarm haha also ahs gbbo final garbage finally find hard fine game furious funny fun action finish air full alone happy almost bb faith cheery character channel change chance arm challenge celtic cause catch cat card car cant cannot candice cancel campaign charge chuckle around clear anyone cool control contact anything anyway con completely complain comedy college coffee coach clown cloud app close cam article fail bitter birthday bill awe awesome awkward bet best believe behind begin bee bed ball become beautiful beat bc bit blame bus block burst burn as assume btw at broadcast bring breezy attempt boy box book avoid body blue blow cover crap crash end email else elate either eat easy earth early dumb dull dry drunk an drug drop bar drink empty enough create entire facebook face eye extra expect exhilarate exhaust excite exact always everything everyone everybody amaze ever event episode dream angry animal dreadful anxiety debate death dear deal dead heard dark damn dad cuz cute customer cunt cry anymore cross answer deep another dm dread draw door animate dollar dog do dismal definitely discourage different difference annoy depression depressing delight hear zero heart sister so snapchat snap smile smh smell sleep sky sit sink somebody single sing since simple sign sick show short she social sometimes shake start straight story store stomach sting still step steal stay star son stand st spend sparkle source sound sorry soon song shall set struggle relentless reverse return result restless rest respond replace remind remember relationship rip rejoice red record realize reality real ready read reach ring rn service saw series send second season scrum school scary scare scar saturday road safe sadness sadly run round rooney room rojo rock stress stuck hearty wake when week website weary wear water watch wall walk voice will video val useless us ur update unhappy understand two who win tv write your young yo yet yesterday year yall ya wrong wow wish worth worst worse worry worried work wonder within with twitter turn style sweet th texan test terrific terrible tell tear team taste swear thanks surprise super sunk summer sulk suggest suffer suck success thank then truly tired true trash track touch top tonight tomorrow together today tire there tip till tiff ticket throw three thousand though tho rd rather rap little yr longer long lonely lol lmao living lively live literally lose listen lip life lie less least learn leadership lay lord loser later mary mess memory member meet medium me may mate match mark lot manchester major madden mad machine ly luck low love laughter late radio horrid important ill idea hurt huff how house hour horror horrible infuriate home holiday hit history hilarity hilarious high hi heyday incense inside last joy lady lack kitchen kiss kik kick key joyous joyful john instagram jesus jazz issue irritation irate iphone involve interview internet mhchat michelle milk paul piss pine picture pic photo peter pessimist person perfect past place part park panic pain others option optimist optimism opportunity pizza play min probably quite queue question push pure punny punch pun process price player pout post positive pop political pm plus please playful open one omg most name my musically music mufc much movie move mourn morning old mope mood month monday momma mom mirth minute mind nd near negative nervous okay ok oh often office nut now note north normal nope night nicole nice ni next news new network able",
    "user get go amp the back like would blue know time love new it start horror look work come turn play girl every fuck good today try year always great this next could hilarious furious say old sorry watch please alarm angry sad happen wish run just think thank kid night shit leave oh burn guy so house move movie player follow us wrong day expect you free late last lol see show one there tonight much many man that tv dreadful nightmare wanna home if rage do month buy call haunt awe bully fire first meet as idea game hilarity dark terrible ticket smile frown ok be wait bring another panic become breezy flight sit away horrible tweet face lip listen animate open absolutely music lmao god match sing relentless remember optimism room nd story serious fume delight together end discourage hear dream week pm put no news th video place grim gbbo top keep boy still gonna felt rejoice hour ball afternoon soon rather episode madden door pessimist crap social film half take safe food write funny ya anything mind kind book dont box foot mean create season scary hey rock step two morning friend early finish pure infuriate long lot depress interview attempt wonder clown fast stop style app your milk voice absolute negative beautiful lady rest nervous make maybe may mate aggravate hope wow rooney air head post lord find block coffee st trash dread deep debate fan swear track dad taste thanks crash forward friday set shock clear shake holiday in short cloud can not mark longer heyday believe stuck star an joyous radio cam dumb series hi steal easy age happy we hell mufc till tire tiff tomorrow minute terrific catch account tho channel animal nice store punch opportunity thousand nut poor campaign celtic loser political album normal surprise ahs park comedy mary cool pine cunt close check floor beat btw round where ill sky fix fill rojo website rd smell jazz email wall folk scrum have twitter scare far part bus yall weekend finger hit kill source win luck rip contact football brown depressing order incense exhilarate punny city pun smh parent saw optimist yeah single eat memory song little enough business imagine cut sometimes finally instead seriously fright anymore who john within let life report yesterday weary super then avoid apparently but fail divide medium due hang all definitely able others trump rap useless pride shoot sure kiss amaze agree promise he me tear anyway date sweet suppose deal dear talk death dangerous decide damn dead often support target dog suck dm suffer dismal suggest different difference die destroy despair deserve depression sulk summer sunk customer cuz cute completely complain texan these college they coach thing though thought threaten class three chuckle chirp con concern conflict term team don cry cross crime tell terror test cover country terrorism cop terrorist control dollar draw success faith fact facebook sound eye extra express experience sparkle exist speak exhaust excite exactly exact everything soul fake everybody fall final some somebody fight fiery someone feeling feel fear favorite something fashion son sort family everyone spend stupid stay sting stomach dull straight dude dry drunk drug drop drive drink stress struggle stuff cheery earth statement ever either event even etc spider entire spirit enjoy stand empty emotion state em else election elate child charlotteprotest cheerfully animosity assume ask what article around arm argument whatever when white anyone whole anxiety answer why at attack well bad want war bc bb bar waste water average awkward awful awesome way weak wear annoy will cheerful with wrath wtf yes afraid affliction adult address add actually action act yet accept yo young ago worth worst america without anger and woman word american world worse also already alone almost worried worry walk wake bed bee card car cant cannot candice cancel towards train treat true truly truth typical burst ugh care carry case tip cheer throw snapchat charlotte charge character change cat chance challenge centre tired cause touch understand unga unhappy biggest black bitter bitch bit birthday bill big blind better bet best vote behind begin blame very brother usa broken broadcast update bright ur break use blood bore boil body val blow bloody sober sleep fine lt positive pout power lose ppl yr president pretty lonely price living lively live probably literally low pop snap lunch picture matter piss pizza plan playful plus manchester point police major policy mad machine ly problem process line light lack quite kitchen king kinda quote rabid kik race kick key racism racist jump rag queue question push pull protest lie levity level provoke less least later learn leadership lead lay laughter laugh pick pic photo must nigga nicole option ni or otherwise never network outrage outside need near pain nation name opinion only online okay office offer offense offend obviously obama now non nothing note omg north on nope my muslim phone musically mistake miss mirth people mine perfect min person might michelle mhchat mess men peter member mom moment momma mourn pak murder pakistan party past paul mother monday most pay mope mood peace money joyful joy joke seem save scar scene goal school gloomy gloom gleeful glee glad give scorn second george secret gotta saturday grieve gun hand sadly hair haha gym sadness guess sam gt grudge growl grow group grind garbage future rule fury forever force for shy sick side sign simple focus since sink sister situation officer smoke forget form shut fucking self send fun fuming sense full fuckin freak sentence service front shall she shot rush ruin join human ready info indian india real reality important realize im really ignore reason idiot record id inside instagram read irritation job jesus rape rapper issue islam irritate insult irate iphone involve intimidate reach internet hurt huff hard how return revenge reverse help revolt hearty heart heard right ring rn road hath hate hat high result restless replace red relationship hot remind horrific horrid resent hillary honestly resentment respect hold respond history zero",
    "user get live watch think worry one know want amaze fear let never horrible even ever anger team life give shock terrible snap service broadcast musically time game ly see the angry unhappy still awful you play what well don customer yet lively someone worst tell bad use actually would anyone my bitter gt guy back afraid rage hold issue part people fuck us god sadly depression quote call best sink bb discourage drop when send point else that take but just least anything put snapchat burst minute deal at another sadness how come win person enough true chance stop probably grudge season lie wrath he lol already they may nothing read news despair straight key feel paul kik please cover almost believe also medium mean struggle dm suck okay sunk horrific be pic last everything within cross road heard she change hear bet success offense side record no care month nicole fuming wow faith level cause involve history maybe nope sign show peter contact final result plus lt fume control zero american texan store truly michelle hand con complain hang happen entire coach lack everybody assume forever year hearty blame action intimidate dismal network group test big will replace respond drug goal return bill low draw price optimism house challenge arm smh card around help experience internet near anymore charge now must cancel email stomach push kitchen thought honestly pine haha worried earth positive garbage important george note cannot rn north completely fail queue mope since exhilarate bore iphone character jesus update have event article thanks hat your name young most quite leadership worth ring death behind biggest st grieve moment avoid scare irate optimist blow perfect useless me business literally heart saturday better top rip pay high emotion damn freak tire voice others very focus reach kind self talk sweet ruin yesterday case light ago dear whatever shy incense single exist less rojo fine fuckin joy dull address leave forget guess everyone horrid lmao hey force for form absolutely hell accept account forward head act annoy haunt hath football hillary heyday foot finally hour find hot horror finger answer hope finish home fire holiday first absolute hit fix flight floor folk follow friday hilarity hilarious food hi free grim friend gloomy affliction girl alone glad all afternoon growl glee age gleeful gloom go gbbo aggravate album gonna grow good alarm gotta agree air ahs great always gun hate full fright hard add happy grind front frown animosity fucking half hair animate america animal and an amp gym fun funny furious fury future adult btw excite anxiety crap country could black cop cool conflict concern comedy blind block college coffee blood bloody clown cloud close bitch crash depress create deep decide debate dead day date dark dangerous dad cuz cute cut cunt cry birthday bit crime clear class city chuckle bright catch cat carry bring car cant candice can campaign cam broken buy brother bus brown burn breezy celtic centre boil chirp blue child body cheery cheerfully cheerful cheer break check book charlotteprotest charlotte box channel boy definitely depressing film as eye extra ask express expect exhaust bully exactly exact every attack etc attempt episode average away enjoy face facebook deserve fact fill fight fiery felt feeling anyway app apparently argument favorite fast fashion far fan family fall fake awe end empty awesome dream dreadful dread door dont dollar dog do become bed bee divide begin different difference die destroy beautiful drink drive bar em awkward election elate either eat ball easy beat bc early dumb due dude dry drunk delight look huff smile song son sometimes something somebody some social sober so smoke smell shoot sleep sky situation sit sister sing simple sick shut shot soon sorry sort soul stupid stuff stuck stress story sting step steal stay statement state start star stand spirit spider spend speak sparkle source sound short shit suffer respect round rooney room rock right revolt reverse revenge restless rest resentment shall resent report remind remember relentless relationship rejoice red reason really rule run rush sad shake set seriously serious series sentence sense seem secret second scrum scorn school scene scary scar say saw save sam safe style suggest reality walk weary wear weak we way water waste war wanna wall wake twitter wait vote video val usa ur unga understand ugh typical website week weekend where yo yes yeah yall ya wtf wrong write worse world work word wonder woman without with wish why whole who white two tweet sulk terrific this thing these there then thank th terrorist terrorism terror term tv tear taste target swear surprise sure suppose support super summer tho though thousand threaten turn try truth trump treat trash train track towards touch tonight tomorrow together today tired tip till tiff ticket throw three realize real human luck mark many manchester man make major madden mad machine lunch love lip lot loser lose lord yr longer long lonely living little mary match mate matter mother morning mood money monday momma mom mistake miss mirth mine mind min milk might mhchat mess men memory member meet listen line move indian islam irritation irritate interview insult instead instagram inside infuriate info india like in imagine im ill ignore if idiot idea id hurt it jazz job john levity learn lead lay laughter laugh later late lady kiss king kinda kill kid kick keep jump joyous joyful joke join mourn movie ready playful ppl power pout post pop poor political policy police pm player party plan place pizza piss picture pick photo phone pessimist peace president pretty pride problem rd rather rapper rape rap rag radio racist racism race rabid question pure punny punch pun pull provoke protest promise process past park much ni obviously obama nut not normal non nightmare night nigga nice next parent new nervous negative need nd nation muslim music murder mufc offend offer office officer panic pakistan pak pain outside outrage otherwise order or option opportunity opinion open only online on omg old ok oh often able"
]

unseeded_model_2 = [
    "user get like make go feel lose day one amp love see thing life anxiety im depression sad even people find need it work much really sadness happy good the today my little right hate look still time say do cry tell think start when take hard want smile stay shake week way night heart someone depress sleep be know something face glee ur year real family literally bit first nightmare you birthday bc pout everyone miss talk tomorrow job we friend cheer twitter new mourn how home lol angry since also either never always though well last nervous leave why end fear so change break post man sting die tear oh and around gloomy school worse worst bad help big sometimes phone long fall walk ask watch question eye thanks ever wait panic person mind yo she sound pretty gbbo lie laughter sober head forget revenge shy understand two wake car best playful fun rag laugh joyful mad seem full world tonight lot weary if half annoy hour dull this attack show alone fact us sparkle body sick hit excite hair hilarious horrid past dream gotta realize without hurt restless cant inside rage wtf gleeful cause whole stand music spend don mhchat piss outside stop age omg eat begin fucking now grim grow scar ready morning bed old keep cheery funny gonna with feeling bitter test lonely already child saw wish learn tip pizza relationship water remember name act ni away wear boy might front fuckin fan adult often depressing min later dog major hope pop offend val cat huff bar process yr mirth red broken mom picture photo dry guess everything that suppose cute second answer close extra damn different son glad finally high somebody bee machine kick kinda ago moment dear fiery death listen sister book pick member living dark who as three college stress summer suggest dude awesome tired awful awkward mood growl elate back drunk reality drink pain favorite empty accept dollar office sulk sure far manchester suffer matter option stuff difference exact exhaust plan mess stupid facebook holiday then instagram cuz kiss throw rapper truth irritation rejoice candice remind write sadly rest chuckle touch shall reverse can lay simple less dead pull read online focus class gym super sit grind song anyway cheerful waste okay monday momma cheerfully sweet future hand history city definitely fine forever experience john bus fill fright joy able win word rap wow reach secret website may hot form memory swear have fashion nigga blow short dread mother what bitch fake perfect mine shot friday beat cut carry men wonder haha absolutely interview situation irate saturday express pic loser easy yet wrong sing country ignore optimism video behind better ok thank deal there these debate officer decide deep th delight texan terrorism deserve despair destroy terror terrific terrible terrorist customer they complain control contact conflict concern con completely comedy cop come tiff coffee coach clown cloud cool could date dismal tho dangerous thought dad thousand threaten cunt cover ticket cross crime create crash crap discourage surprise divide enough episode etc store event stomach every everybody step steal statement exactly state star exhilarate exist expect st spirit spider speak source soul fail faith sort sorry soon some fast entire enjoy dm story term team taste target dont door draw dreadful clear drive drop drug support sunk suck due success dumb early earth style stuck struggle election else em email emotion straight till charlotte tire apparently assume within woman article arm argument app attempt anything anyone anymore worried worry another at average believe bb wall wanna become beautiful war weak ball avoid weekend whatever where white awe will worth animosity animate address yes aggravate yesterday afternoon afraid affliction add animal actually action young account your absolute agree ahs air alarm album all almost yeah yall ya amaze america american wrath an anger would vote voice together try truly catch trump case care card turn centre cannot tv cancel tweet campaign cam celtic challenge bet treat top chirp towards track train trash check chance charlotteprotest snapchat charge character channel true call buy but blind boil usa blue bloody blood block blame business black use useless very bill biggest update bore box unhappy unga breezy bright bring broadcast ugh brother brown btw bully burn burst typical social fire snap lively longer ppl president price lmao pride live lord probably problem promise lip line protest power positive provoke police many please plus pm point madden policy poor ly lunch luck lt low political light pun mary kid race king racism kind kill kik racist rabid key radio just jump joyous rape kitchen lack levity leadership level let punch punny least pure lead lady push put queue quite late quote mark match smoke or nice opportunity next news optimist network negative opinion order near nd nation others otherwise nicole open muslim only offer offense on obviously obama nut nothing no note not north normal nope non must musically mate place minute pessimist peter milk michelle pine play mistake meet medium mean me maybe player peace pay outrage most murder mufc pak movie move pakistan parent paul mope park month money part party joke join rather series gloom send sense sentence give girl george goal serious garbage game seriously fury furious self god set scare sam guy gun gt grudge save group season scary scene grieve great scorn scrum service fuming rd sink floor flight fix single finish finger sky follow final film smell fight smh felt folk food fume free shit shock shoot fuck frown shut freak foot forward side sign force for football safe rush run report incense in important imagine replace ill idiot indian idea id resent human resentment respect india info hang iphone jesus jazz issue islam reason irritate involve infuriate intimidate internet insult instead record relentless house respond horror he hell hearty room heard hear rooney haunt horrific hath round hat ruin rule happen rojo hey heyday hi rock road hilarity hillary rn rip hold ring revolt honestly return horrible result zero",
    "user people like say terrorism fuck really amp black outrage terror trump way make need want take get think try offend use war you pakistan anger the let police bully look word fury it if hell go not kill die woman give fear nothing insult problem serious right would no talk he this they and awful irritate someone pay ask white joke man world we country but shit sad fight attack state stop well something yes protest seriously concern all support fume boil shoot blood racism don revenge never india why vote hillary speak destroy yeah can hope terrorist threaten money sure human king keep afraid thing reason what mean know men gun better drive sort phone provoke cop that racist america tweet towards away point wait pretty decide date rabid break could resent weak hand even might soul rule miss call bitch come future class truth ppl some shot party levity care suppose maybe wanna on rag child cheer etc help force crime bright charlotte ugh without fuming these power be hold fact instead line sober president train side scene muslim centre sense islam mistake usa join unga peace despair whole day only race election must obviously respect agree nation jump mine resentment fake in brother bad chirp indian charlotteprotest many opinion lead every officer or pak save affliction gloom rush kinda conflict non order obama situation promise stuff shut bloody address exactly honestly deserve info for murder blind ignore policy animosity revolt stupid target offense offer spider spirit parent em lunch outside idiot id statement big rape dangerous american biggest leave guy average otherwise typical form scorn fiery wrath hath treat add sam smoke imagine argument carry express sentence enjoy term happen self seem due where level hot poor horrific moment freak divide shock bore whatever just ruin back quote shy pull growl exist pride put matter online cut intimidate depress very dont case nightmare dude brown nigga city hey front plan mother light report apparently check see cheerfully business guess act thought fashion emotion buy kind pick laugh accept secret experience lt finger weekend fire football waste wtf gym im broken job kid rage rapper grudge good cheerful time feeling gloomy hate felt gleeful forward glee already afternoon fill hat glad age film girl gt feel ago goal head fall grim grieve family absolute grind great gotta group fan absolutely grow far have haunt gonna god fast favorite adult aggravate actually george folk first fuckin fix album flight floor frown focus half follow forget food foot fright friend friday hang free account forever fucking hair alarm haha also ahs gbbo final garbage finally find hard fine game furious funny fun action finish air full alone happy almost bb faith cheery character channel change chance arm challenge celtic cause catch cat card car cant cannot candice cancel campaign charge chuckle around clear anyone cool control contact anything anyway con completely complain comedy college coffee coach clown cloud app close cam article fail bitter birthday bill awe awesome awkward bet best believe behind begin bee bed ball become beautiful beat bc bit blame bus block burst burn as assume btw at broadcast bring breezy attempt boy box book avoid body blue blow cover crap crash end email else elate either eat easy earth early dumb dull dry drunk an drug drop bar drink empty enough create entire facebook face eye extra expect exhilarate exhaust excite exact always everything everyone everybody amaze ever event episode dream angry animal dreadful anxiety debate death dear deal dead heard dark damn dad cuz cute customer cunt cry anymore cross answer deep another dm dread draw door animate dollar dog do dismal definitely discourage different difference annoy depression depressing delight hear zero heart sister so snapchat snap smile smh smell sleep sky sit sink somebody single sing since simple sign sick show short she social sometimes shake start straight story store stomach sting still step steal stay star son stand st spend sparkle source sound sorry soon song shall set struggle relentless reverse return result restless rest respond replace remind remember relationship rip rejoice red record realize reality real ready read reach ring rn service saw series send second season scrum school scary scare scar saturday road safe sadness sadly run round rooney room rojo rock stress stuck hearty wake when week website weary wear water watch wall walk voice will video val useless us ur update unhappy understand two who win tv write your young yo yet yesterday year yall ya wrong wow wish worth worst worse worry worried work wonder within with twitter turn style sweet th texan test terrific terrible tell tear team taste swear thanks surprise super sunk summer sulk suggest suffer suck success thank then truly tired true trash track touch top tonight tomorrow together today tire there tip till tiff ticket throw three thousand though tho rd rather rap little yr longer long lonely lol lmao living lively live literally lose listen lip life lie less least learn leadership lay lord loser later mary mess memory member meet medium me may mate match mark lot manchester major madden mad machine ly luck low love laughter late radio horrid important ill idea hurt huff how house hour horror horrible infuriate home holiday hit history hilarity hilarious high hi heyday incense inside last joy lady lack kitchen kiss kik kick key joyous joyful john instagram jesus jazz issue irritation irate iphone involve interview internet mhchat michelle milk paul piss pine picture pic photo peter pessimist person perfect past place part park panic pain others option optimist optimism opportunity pizza play min probably quite queue question push pure punny punch pun process price player pout post positive pop political pm plus please playful open one omg most name my musically music mufc much movie move mourn morning old mope mood month monday momma mom mirth minute mind nd near negative nervous okay ok oh often office nut now note north normal nope night nicole nice ni next news new network able",
    "user get go amp the back like would blue know time love new it start horror look work come turn play girl every fuck good today try year always great this next could hilarious furious say old sorry watch please alarm angry sad happen wish run just think thank kid night shit leave oh burn guy so house move movie player follow us wrong day expect you free late last lol see show one there tonight much many man that tv dreadful nightmare wanna home if rage do month buy call haunt awe bully fire first meet as idea game hilarity dark terrible ticket smile frown ok be wait bring another panic become breezy flight sit away horrible tweet face lip listen animate open absolutely music lmao god match sing relentless remember optimism room nd story serious fume delight together end discourage hear dream week pm put no news th video place grim gbbo top keep boy still gonna felt rejoice hour ball afternoon soon rather episode madden door pessimist crap social film half take safe food write funny ya anything mind kind book dont box foot mean create season scary hey rock step two morning friend early finish pure infuriate long lot depress interview attempt wonder clown fast stop style app your milk voice absolute negative beautiful lady rest nervous make maybe may mate aggravate hope wow rooney air head post lord find block coffee st trash dread deep debate fan swear track dad taste thanks crash forward friday set shock clear shake holiday in short cloud can not mark longer heyday believe stuck star an joyous radio cam dumb series hi steal easy age happy we hell mufc till tire tiff tomorrow minute terrific catch account tho channel animal nice store punch opportunity thousand nut poor campaign celtic loser political album normal surprise ahs park comedy mary cool pine cunt close check floor beat btw round where ill sky fix fill rojo website rd smell jazz email wall folk scrum have twitter scare far part bus yall weekend finger hit kill source win luck rip contact football brown depressing order incense exhilarate punny city pun smh parent saw optimist yeah single eat memory song little enough business imagine cut sometimes finally instead seriously fright anymore who john within let life report yesterday weary super then avoid apparently but fail divide medium due hang all definitely able others trump rap useless pride shoot sure kiss amaze agree promise he me tear anyway date sweet suppose deal dear talk death dangerous decide damn dead often support target dog suck dm suffer dismal suggest different difference die destroy despair deserve depression sulk summer sunk customer cuz cute completely complain texan these college they coach thing though thought threaten class three chuckle chirp con concern conflict term team don cry cross crime tell terror test cover country terrorism cop terrorist control dollar draw success faith fact facebook sound eye extra express experience sparkle exist speak exhaust excite exactly exact everything soul fake everybody fall final some somebody fight fiery someone feeling feel fear favorite something fashion son sort family everyone spend stupid stay sting stomach dull straight dude dry drunk drug drop drive drink stress struggle stuff cheery earth statement ever either event even etc spider entire spirit enjoy stand empty emotion state em else election elate child charlotteprotest cheerfully animosity assume ask what article around arm argument whatever when white anyone whole anxiety answer why at attack well bad want war bc bb bar waste water average awkward awful awesome way weak wear annoy will cheerful with wrath wtf yes afraid affliction adult address add actually action act yet accept yo young ago worth worst america without anger and woman word american world worse also already alone almost worried worry walk wake bed bee card car cant cannot candice cancel towards train treat true truly truth typical burst ugh care carry case tip cheer throw snapchat charlotte charge character change cat chance challenge centre tired cause touch understand unga unhappy biggest black bitter bitch bit birthday bill big blind better bet best vote behind begin blame very brother usa broken broadcast update bright ur break use blood bore boil body val blow bloody sober sleep fine lt positive pout power lose ppl yr president pretty lonely price living lively live probably literally low pop snap lunch picture matter piss pizza plan playful plus manchester point police major policy mad machine ly problem process line light lack quite kitchen king kinda quote rabid kik race kick key racism racist jump rag queue question push pull protest lie levity level provoke less least later learn leadership lead lay laughter laugh pick pic photo must nigga nicole option ni or otherwise never network outrage outside need near pain nation name opinion only online okay office offer offense offend obviously obama now non nothing note omg north on nope my muslim phone musically mistake miss mirth people mine perfect min person might michelle mhchat mess men peter member mom moment momma mourn pak murder pakistan party past paul mother monday most pay mope mood peace money joyful joy joke seem save scar scene goal school gloomy gloom gleeful glee glad give scorn second george secret gotta saturday grieve gun hand sadly hair haha gym sadness guess sam gt grudge growl grow group grind garbage future rule fury forever force for shy sick side sign simple focus since sink sister situation officer smoke forget form shut fucking self send fun fuming sense full fuckin freak sentence service front shall she shot rush ruin join human ready info indian india real reality important realize im really ignore reason idiot record id inside instagram read irritation job jesus rape rapper issue islam irritate insult irate iphone involve intimidate reach internet hurt huff hard how return revenge reverse help revolt hearty heart heard right ring rn road hath hate hat high result restless replace red relationship hot remind horrific horrid resent hillary honestly resentment respect hold respond history zero",
    "user get live watch think worry one know want amaze fear let never horrible even ever anger team life give shock terrible snap service broadcast musically time game ly see the angry unhappy still awful you play what well don customer yet lively someone worst tell bad use actually would anyone my bitter gt guy back afraid rage hold issue part people fuck us god sadly depression quote call best sink bb discourage drop when send point else that take but just least anything put snapchat burst minute deal at another sadness how come win person enough true chance stop probably grudge season lie wrath he lol already they may nothing read news despair straight key feel paul kik please cover almost believe also medium mean struggle dm suck okay sunk horrific be pic last everything within cross road heard she change hear bet success offense side record no care month nicole fuming wow faith level cause involve history maybe nope sign show peter contact final result plus lt fume control zero american texan store truly michelle hand con complain hang happen entire coach lack everybody assume forever year hearty blame action intimidate dismal network group test big will replace respond drug goal return bill low draw price optimism house challenge arm smh card around help experience internet near anymore charge now must cancel email stomach push kitchen thought honestly pine haha worried earth positive garbage important george note cannot rn north completely fail queue mope since exhilarate bore iphone character jesus update have event article thanks hat your name young most quite leadership worth ring death behind biggest st grieve moment avoid scare irate optimist blow perfect useless me business literally heart saturday better top rip pay high emotion damn freak tire voice others very focus reach kind self talk sweet ruin yesterday case light ago dear whatever shy incense single exist less rojo fine fuckin joy dull address leave forget guess everyone horrid lmao hey force for form absolutely hell accept account forward head act annoy haunt hath football hillary heyday foot finally hour find hot horror finger answer hope finish home fire holiday first absolute hit fix flight floor folk follow friday hilarity hilarious food hi free grim friend gloomy affliction girl alone glad all afternoon growl glee age gleeful gloom go gbbo aggravate album gonna grow good alarm gotta agree air ahs great always gun hate full fright hard add happy grind front frown animosity fucking half hair animate america animal and an amp gym fun funny furious fury future adult btw excite anxiety crap country could black cop cool conflict concern comedy blind block college coffee blood bloody clown cloud close bitch crash depress create deep decide debate dead day date dark dangerous dad cuz cute cut cunt cry birthday bit crime clear class city chuckle bright catch cat carry bring car cant candice can campaign cam broken buy brother bus brown burn breezy celtic centre boil chirp blue child body cheery cheerfully cheerful cheer break check book charlotteprotest charlotte box channel boy definitely depressing film as eye extra ask express expect exhaust bully exactly exact every attack etc attempt episode average away enjoy face facebook deserve fact fill fight fiery felt feeling anyway app apparently argument favorite fast fashion far fan family fall fake awe end empty awesome dream dreadful dread door dont dollar dog do become bed bee divide begin different difference die destroy beautiful drink drive bar em awkward election elate either eat ball easy beat bc early dumb due dude dry drunk delight look huff smile song son sometimes something somebody some social sober so smoke smell shoot sleep sky situation sit sister sing simple sick shut shot soon sorry sort soul stupid stuff stuck stress story sting step steal stay statement state start star stand spirit spider spend speak sparkle source sound short shit suffer respect round rooney room rock right revolt reverse revenge restless rest resentment shall resent report remind remember relentless relationship rejoice red reason really rule run rush sad shake set seriously serious series sentence sense seem secret second scrum scorn school scene scary scar say saw save sam safe style suggest reality walk weary wear weak we way water waste war wanna wall wake twitter wait vote video val usa ur unga understand ugh typical website week weekend where yo yes yeah yall ya wtf wrong write worse world work word wonder woman without with wish why whole who white two tweet sulk terrific this thing these there then thank th terrorist terrorism terror term tv tear taste target swear surprise sure suppose support super summer tho though thousand threaten turn try truth trump treat trash train track towards touch tonight tomorrow together today tired tip till tiff ticket throw three realize real human luck mark many manchester man make major madden mad machine lunch love lip lot loser lose lord yr longer long lonely living little mary match mate matter mother morning mood money monday momma mom mistake miss mirth mine mind min milk might mhchat mess men memory member meet listen line move indian islam irritation irritate interview insult instead instagram inside infuriate info india like in imagine im ill ignore if idiot idea id hurt it jazz job john levity learn lead lay laughter laugh later late lady kiss king kinda kill kid kick keep jump joyous joyful joke join mourn movie ready playful ppl power pout post pop poor political policy police pm player party plan place pizza piss picture pick photo phone pessimist peace president pretty pride problem rd rather rapper rape rap rag radio racist racism race rabid question pure punny punch pun pull provoke protest promise process past park much ni obviously obama nut not normal non nightmare night nigga nice next parent new nervous negative need nd nation muslim music murder mufc offend offer office officer panic pakistan pak pain outside outrage otherwise order or option opportunity opinion open only online on omg old ok oh often able"
]

unseeded_model_3 = [
    "user get like make go feel lose day one amp love see thing life anxiety im depression sad even people find need it work much really sadness happy good the today my little right hate look still time say do cry tell think start when take hard want smile stay shake week way night heart someone depress sleep be know something face glee ur year real family literally bit first nightmare you birthday bc pout everyone miss talk tomorrow job we friend cheer twitter new mourn how home lol angry since also either never always though well last nervous leave why end fear so change break post man sting die tear oh and around gloomy school worse worst bad help big sometimes phone long fall walk ask watch question eye thanks ever wait panic person mind yo she sound pretty gbbo lie laughter sober head forget revenge shy understand two wake car best playful fun rag laugh joyful mad seem full world tonight lot weary if half annoy hour dull this attack show alone fact us sparkle body sick hit excite hair hilarious horrid past dream gotta realize without hurt restless cant inside rage wtf gleeful cause whole stand music spend don mhchat piss outside stop age omg eat begin fucking now grim grow scar ready morning bed old keep cheery funny gonna with feeling bitter test lonely already child saw wish learn tip pizza relationship water remember name act ni away wear boy might front fuckin fan adult often depressing min later dog major hope pop offend val cat huff bar process yr mirth red broken mom picture photo dry guess everything that suppose cute second answer close extra damn different son glad finally high somebody bee machine kick kinda ago moment dear fiery death listen sister book pick member living dark who as three college stress summer suggest dude awesome tired awful awkward mood growl elate back drunk reality drink pain favorite empty accept dollar office sulk sure far manchester suffer matter option stuff difference exact exhaust plan mess stupid facebook holiday then instagram cuz kiss throw rapper truth irritation rejoice candice remind write sadly rest chuckle touch shall reverse can lay simple less dead pull read online focus class gym super sit grind song anyway cheerful waste okay monday momma cheerfully sweet future hand history city definitely fine forever experience john bus fill fright joy able win word rap wow reach secret website may hot form memory swear have fashion nigga blow short dread mother what bitch fake perfect mine shot friday beat cut carry men wonder haha absolutely interview situation irate saturday express pic loser easy yet wrong sing country ignore optimism video behind better ok thank deal there these debate officer decide deep th delight texan terrorism deserve despair destroy terror terrific terrible terrorist customer they complain control contact conflict concern con completely comedy cop come tiff coffee coach clown cloud cool could date dismal tho dangerous thought dad thousand threaten cunt cover ticket cross crime create crash crap discourage surprise divide enough episode etc store event stomach every everybody step steal statement exactly state star exhilarate exist expect st spirit spider speak source soul fail faith sort sorry soon some fast entire enjoy dm story term team taste target dont door draw dreadful clear drive drop drug support sunk suck due success dumb early earth style stuck struggle election else em email emotion straight till charlotte tire apparently assume within woman article arm argument app attempt anything anyone anymore worried worry another at average believe bb wall wanna become beautiful war weak ball avoid weekend whatever where white awe will worth animosity animate address yes aggravate yesterday afternoon afraid affliction add animal actually action young account your absolute agree ahs air alarm album all almost yeah yall ya amaze america american wrath an anger would vote voice together try truly catch trump case care card turn centre cannot tv cancel tweet campaign cam celtic challenge bet treat top chirp towards track train trash check chance charlotteprotest snapchat charge character channel true call buy but blind boil usa blue bloody blood block blame business black use useless very bill biggest update bore box unhappy unga breezy bright bring broadcast ugh brother brown btw bully burn burst typical social fire snap lively longer ppl president price lmao pride live lord probably problem promise lip line protest power positive provoke police many please plus pm point madden policy poor ly lunch luck lt low political light pun mary kid race king racism kind kill kik racist rabid key radio just jump joyous rape kitchen lack levity leadership level let punch punny least pure lead lady push put queue quite late quote mark match smoke or nice opportunity next news optimist network negative opinion order near nd nation others otherwise nicole open muslim only offer offense on obviously obama nut nothing no note not north normal nope non must musically mate place minute pessimist peter milk michelle pine play mistake meet medium mean me maybe player peace pay outrage most murder mufc pak movie move pakistan parent paul mope park month money part party joke join rather series gloom send sense sentence give girl george goal serious garbage game seriously fury furious self god set scare sam guy gun gt grudge save group season scary scene grieve great scorn scrum service fuming rd sink floor flight fix single finish finger sky follow final film smell fight smh felt folk food fume free shit shock shoot fuck frown shut freak foot forward side sign force for football safe rush run report incense in important imagine replace ill idiot indian idea id resent human resentment respect india info hang iphone jesus jazz issue islam reason irritate involve infuriate intimidate internet insult instead record relentless house respond horror he hell hearty room heard hear rooney haunt horrific hath round hat ruin rule happen rojo hey heyday hi rock road hilarity hillary rn rip hold ring revolt honestly return horrible result zero",
    "user people like say terrorism fuck really amp black outrage terror trump way make need want take get think try offend use war you pakistan anger the let police bully look word fury it if hell go not kill die woman give fear nothing insult problem serious right would no talk he this they and awful irritate someone pay ask white joke man world we country but shit sad fight attack state stop well something yes protest seriously concern all support fume boil shoot blood racism don revenge never india why vote hillary speak destroy yeah can hope terrorist threaten money sure human king keep afraid thing reason what mean know men gun better drive sort phone provoke cop that racist america tweet towards away point wait pretty decide date rabid break could resent weak hand even might soul rule miss call bitch come future class truth ppl some shot party levity care suppose maybe wanna on rag child cheer etc help force crime bright charlotte ugh without fuming these power be hold fact instead line sober president train side scene muslim centre sense islam mistake usa join unga peace despair whole day only race election must obviously respect agree nation jump mine resentment fake in brother bad chirp indian charlotteprotest many opinion lead every officer or pak save affliction gloom rush kinda conflict non order obama situation promise stuff shut bloody address exactly honestly deserve info for murder blind ignore policy animosity revolt stupid target offense offer spider spirit parent em lunch outside idiot id statement big rape dangerous american biggest leave guy average otherwise typical form scorn fiery wrath hath treat add sam smoke imagine argument carry express sentence enjoy term happen self seem due where level hot poor horrific moment freak divide shock bore whatever just ruin back quote shy pull growl exist pride put matter online cut intimidate depress very dont case nightmare dude brown nigga city hey front plan mother light report apparently check see cheerfully business guess act thought fashion emotion buy kind pick laugh accept secret experience lt finger weekend fire football waste wtf gym im broken job kid rage rapper grudge good cheerful time feeling gloomy hate felt gleeful forward glee already afternoon fill hat glad age film girl gt feel ago goal head fall grim grieve family absolute grind great gotta group fan absolutely grow far have haunt gonna god fast favorite adult aggravate actually george folk first fuckin fix album flight floor frown focus half follow forget food foot fright friend friday hang free account forever fucking hair alarm haha also ahs gbbo final garbage finally find hard fine game furious funny fun action finish air full alone happy almost bb faith cheery character channel change chance arm challenge celtic cause catch cat card car cant cannot candice cancel campaign charge chuckle around clear anyone cool control contact anything anyway con completely complain comedy college coffee coach clown cloud app close cam article fail bitter birthday bill awe awesome awkward bet best believe behind begin bee bed ball become beautiful beat bc bit blame bus block burst burn as assume btw at broadcast bring breezy attempt boy box book avoid body blue blow cover crap crash end email else elate either eat easy earth early dumb dull dry drunk an drug drop bar drink empty enough create entire facebook face eye extra expect exhilarate exhaust excite exact always everything everyone everybody amaze ever event episode dream angry animal dreadful anxiety debate death dear deal dead heard dark damn dad cuz cute customer cunt cry anymore cross answer deep another dm dread draw door animate dollar dog do dismal definitely discourage different difference annoy depression depressing delight hear zero heart sister so snapchat snap smile smh smell sleep sky sit sink somebody single sing since simple sign sick show short she social sometimes shake start straight story store stomach sting still step steal stay star son stand st spend sparkle source sound sorry soon song shall set struggle relentless reverse return result restless rest respond replace remind remember relationship rip rejoice red record realize reality real ready read reach ring rn service saw series send second season scrum school scary scare scar saturday road safe sadness sadly run round rooney room rojo rock stress stuck hearty wake when week website weary wear water watch wall walk voice will video val useless us ur update unhappy understand two who win tv write your young yo yet yesterday year yall ya wrong wow wish worth worst worse worry worried work wonder within with twitter turn style sweet th texan test terrific terrible tell tear team taste swear thanks surprise super sunk summer sulk suggest suffer suck success thank then truly tired true trash track touch top tonight tomorrow together today tire there tip till tiff ticket throw three thousand though tho rd rather rap little yr longer long lonely lol lmao living lively live literally lose listen lip life lie less least learn leadership lay lord loser later mary mess memory member meet medium me may mate match mark lot manchester major madden mad machine ly luck low love laughter late radio horrid important ill idea hurt huff how house hour horror horrible infuriate home holiday hit history hilarity hilarious high hi heyday incense inside last joy lady lack kitchen kiss kik kick key joyous joyful john instagram jesus jazz issue irritation irate iphone involve interview internet mhchat michelle milk paul piss pine picture pic photo peter pessimist person perfect past place part park panic pain others option optimist optimism opportunity pizza play min probably quite queue question push pure punny punch pun process price player pout post positive pop political pm plus please playful open one omg most name my musically music mufc much movie move mourn morning old mope mood month monday momma mom mirth minute mind nd near negative nervous okay ok oh often office nut now note north normal nope night nicole nice ni next news new network able",
    "user get go amp the back like would blue know time love new it start horror look work come turn play girl every fuck good today try year always great this next could hilarious furious say old sorry watch please alarm angry sad happen wish run just think thank kid night shit leave oh burn guy so house move movie player follow us wrong day expect you free late last lol see show one there tonight much many man that tv dreadful nightmare wanna home if rage do month buy call haunt awe bully fire first meet as idea game hilarity dark terrible ticket smile frown ok be wait bring another panic become breezy flight sit away horrible tweet face lip listen animate open absolutely music lmao god match sing relentless remember optimism room nd story serious fume delight together end discourage hear dream week pm put no news th video place grim gbbo top keep boy still gonna felt rejoice hour ball afternoon soon rather episode madden door pessimist crap social film half take safe food write funny ya anything mind kind book dont box foot mean create season scary hey rock step two morning friend early finish pure infuriate long lot depress interview attempt wonder clown fast stop style app your milk voice absolute negative beautiful lady rest nervous make maybe may mate aggravate hope wow rooney air head post lord find block coffee st trash dread deep debate fan swear track dad taste thanks crash forward friday set shock clear shake holiday in short cloud can not mark longer heyday believe stuck star an joyous radio cam dumb series hi steal easy age happy we hell mufc till tire tiff tomorrow minute terrific catch account tho channel animal nice store punch opportunity thousand nut poor campaign celtic loser political album normal surprise ahs park comedy mary cool pine cunt close check floor beat btw round where ill sky fix fill rojo website rd smell jazz email wall folk scrum have twitter scare far part bus yall weekend finger hit kill source win luck rip contact football brown depressing order incense exhilarate punny city pun smh parent saw optimist yeah single eat memory song little enough business imagine cut sometimes finally instead seriously fright anymore who john within let life report yesterday weary super then avoid apparently but fail divide medium due hang all definitely able others trump rap useless pride shoot sure kiss amaze agree promise he me tear anyway date sweet suppose deal dear talk death dangerous decide damn dead often support target dog suck dm suffer dismal suggest different difference die destroy despair deserve depression sulk summer sunk customer cuz cute completely complain texan these college they coach thing though thought threaten class three chuckle chirp con concern conflict term team don cry cross crime tell terror test cover country terrorism cop terrorist control dollar draw success faith fact facebook sound eye extra express experience sparkle exist speak exhaust excite exactly exact everything soul fake everybody fall final some somebody fight fiery someone feeling feel fear favorite something fashion son sort family everyone spend stupid stay sting stomach dull straight dude dry drunk drug drop drive drink stress struggle stuff cheery earth statement ever either event even etc spider entire spirit enjoy stand empty emotion state em else election elate child charlotteprotest cheerfully animosity assume ask what article around arm argument whatever when white anyone whole anxiety answer why at attack well bad want war bc bb bar waste water average awkward awful awesome way weak wear annoy will cheerful with wrath wtf yes afraid affliction adult address add actually action act yet accept yo young ago worth worst america without anger and woman word american world worse also already alone almost worried worry walk wake bed bee card car cant cannot candice cancel towards train treat true truly truth typical burst ugh care carry case tip cheer throw snapchat charlotte charge character change cat chance challenge centre tired cause touch understand unga unhappy biggest black bitter bitch bit birthday bill big blind better bet best vote behind begin blame very brother usa broken broadcast update bright ur break use blood bore boil body val blow bloody sober sleep fine lt positive pout power lose ppl yr president pretty lonely price living lively live probably literally low pop snap lunch picture matter piss pizza plan playful plus manchester point police major policy mad machine ly problem process line light lack quite kitchen king kinda quote rabid kik race kick key racism racist jump rag queue question push pull protest lie levity level provoke less least later learn leadership lead lay laughter laugh pick pic photo must nigga nicole option ni or otherwise never network outrage outside need near pain nation name opinion only online okay office offer offense offend obviously obama now non nothing note omg north on nope my muslim phone musically mistake miss mirth people mine perfect min person might michelle mhchat mess men peter member mom moment momma mourn pak murder pakistan party past paul mother monday most pay mope mood peace money joyful joy joke seem save scar scene goal school gloomy gloom gleeful glee glad give scorn second george secret gotta saturday grieve gun hand sadly hair haha gym sadness guess sam gt grudge growl grow group grind garbage future rule fury forever force for shy sick side sign simple focus since sink sister situation officer smoke forget form shut fucking self send fun fuming sense full fuckin freak sentence service front shall she shot rush ruin join human ready info indian india real reality important realize im really ignore reason idiot record id inside instagram read irritation job jesus rape rapper issue islam irritate insult irate iphone involve intimidate reach internet hurt huff hard how return revenge reverse help revolt hearty heart heard right ring rn road hath hate hat high result restless replace red relationship hot remind horrific horrid resent hillary honestly resentment respect hold respond history zero",
    "user get live watch think worry one know want amaze fear let never horrible even ever anger team life give shock terrible snap service broadcast musically time game ly see the angry unhappy still awful you play what well don customer yet lively someone worst tell bad use actually would anyone my bitter gt guy back afraid rage hold issue part people fuck us god sadly depression quote call best sink bb discourage drop when send point else that take but just least anything put snapchat burst minute deal at another sadness how come win person enough true chance stop probably grudge season lie wrath he lol already they may nothing read news despair straight key feel paul kik please cover almost believe also medium mean struggle dm suck okay sunk horrific be pic last everything within cross road heard she change hear bet success offense side record no care month nicole fuming wow faith level cause involve history maybe nope sign show peter contact final result plus lt fume control zero american texan store truly michelle hand con complain hang happen entire coach lack everybody assume forever year hearty blame action intimidate dismal network group test big will replace respond drug goal return bill low draw price optimism house challenge arm smh card around help experience internet near anymore charge now must cancel email stomach push kitchen thought honestly pine haha worried earth positive garbage important george note cannot rn north completely fail queue mope since exhilarate bore iphone character jesus update have event article thanks hat your name young most quite leadership worth ring death behind biggest st grieve moment avoid scare irate optimist blow perfect useless me business literally heart saturday better top rip pay high emotion damn freak tire voice others very focus reach kind self talk sweet ruin yesterday case light ago dear whatever shy incense single exist less rojo fine fuckin joy dull address leave forget guess everyone horrid lmao hey force for form absolutely hell accept account forward head act annoy haunt hath football hillary heyday foot finally hour find hot horror finger answer hope finish home fire holiday first absolute hit fix flight floor folk follow friday hilarity hilarious food hi free grim friend gloomy affliction girl alone glad all afternoon growl glee age gleeful gloom go gbbo aggravate album gonna grow good alarm gotta agree air ahs great always gun hate full fright hard add happy grind front frown animosity fucking half hair animate america animal and an amp gym fun funny furious fury future adult btw excite anxiety crap country could black cop cool conflict concern comedy blind block college coffee blood bloody clown cloud close bitch crash depress create deep decide debate dead day date dark dangerous dad cuz cute cut cunt cry birthday bit crime clear class city chuckle bright catch cat carry bring car cant candice can campaign cam broken buy brother bus brown burn breezy celtic centre boil chirp blue child body cheery cheerfully cheerful cheer break check book charlotteprotest charlotte box channel boy definitely depressing film as eye extra ask express expect exhaust bully exactly exact every attack etc attempt episode average away enjoy face facebook deserve fact fill fight fiery felt feeling anyway app apparently argument favorite fast fashion far fan family fall fake awe end empty awesome dream dreadful dread door dont dollar dog do become bed bee divide begin different difference die destroy beautiful drink drive bar em awkward election elate either eat ball easy beat bc early dumb due dude dry drunk delight look huff smile song son sometimes something somebody some social sober so smoke smell shoot sleep sky situation sit sister sing simple sick shut shot soon sorry sort soul stupid stuff stuck stress story sting step steal stay statement state start star stand spirit spider spend speak sparkle source sound short shit suffer respect round rooney room rock right revolt reverse revenge restless rest resentment shall resent report remind remember relentless relationship rejoice red reason really rule run rush sad shake set seriously serious series sentence sense seem secret second scrum scorn school scene scary scar say saw save sam safe style suggest reality walk weary wear weak we way water waste war wanna wall wake twitter wait vote video val usa ur unga understand ugh typical website week weekend where yo yes yeah yall ya wtf wrong write worse world work word wonder woman without with wish why whole who white two tweet sulk terrific this thing these there then thank th terrorist terrorism terror term tv tear taste target swear surprise sure suppose support super summer tho though thousand threaten turn try truth trump treat trash train track towards touch tonight tomorrow together today tired tip till tiff ticket throw three realize real human luck mark many manchester man make major madden mad machine lunch love lip lot loser lose lord yr longer long lonely living little mary match mate matter mother morning mood money monday momma mom mistake miss mirth mine mind min milk might mhchat mess men memory member meet listen line move indian islam irritation irritate interview insult instead instagram inside infuriate info india like in imagine im ill ignore if idiot idea id hurt it jazz job john levity learn lead lay laughter laugh later late lady kiss king kinda kill kid kick keep jump joyous joyful joke join mourn movie ready playful ppl power pout post pop poor political policy police pm player party plan place pizza piss picture pick photo phone pessimist peace president pretty pride problem rd rather rapper rape rap rag radio racist racism race rabid question pure punny punch pun pull provoke protest promise process past park much ni obviously obama nut not normal non nightmare night nigga nice next parent new nervous negative need nd nation muslim music murder mufc offend offer office officer panic pakistan pak pain outside outrage otherwise order or option opportunity opinion open only online on omg old ok oh often able"
]

unseeded_model_4 = [
    "user get like make go feel lose day one amp love see thing life anxiety im depression sad even people find need it work much really sadness happy good the today my little right hate look still time say do cry tell think start when take hard want smile stay shake week way night heart someone depress sleep be know something face glee ur year real family literally bit first nightmare you birthday bc pout everyone miss talk tomorrow job we friend cheer twitter new mourn how home lol angry since also either never always though well last nervous leave why end fear so change break post man sting die tear oh and around gloomy school worse worst bad help big sometimes phone long fall walk ask watch question eye thanks ever wait panic person mind yo she sound pretty gbbo lie laughter sober head forget revenge shy understand two wake car best playful fun rag laugh joyful mad seem full world tonight lot weary if half annoy hour dull this attack show alone fact us sparkle body sick hit excite hair hilarious horrid past dream gotta realize without hurt restless cant inside rage wtf gleeful cause whole stand music spend don mhchat piss outside stop age omg eat begin fucking now grim grow scar ready morning bed old keep cheery funny gonna with feeling bitter test lonely already child saw wish learn tip pizza relationship water remember name act ni away wear boy might front fuckin fan adult often depressing min later dog major hope pop offend val cat huff bar process yr mirth red broken mom picture photo dry guess everything that suppose cute second answer close extra damn different son glad finally high somebody bee machine kick kinda ago moment dear fiery death listen sister book pick member living dark who as three college stress summer suggest dude awesome tired awful awkward mood growl elate back drunk reality drink pain favorite empty accept dollar office sulk sure far manchester suffer matter option stuff difference exact exhaust plan mess stupid facebook holiday then instagram cuz kiss throw rapper truth irritation rejoice candice remind write sadly rest chuckle touch shall reverse can lay simple less dead pull read online focus class gym super sit grind song anyway cheerful waste okay monday momma cheerfully sweet future hand history city definitely fine forever experience john bus fill fright joy able win word rap wow reach secret website may hot form memory swear have fashion nigga blow short dread mother what bitch fake perfect mine shot friday beat cut carry men wonder haha absolutely interview situation irate saturday express pic loser easy yet wrong sing country ignore optimism video behind better ok thank deal there these debate officer decide deep th delight texan terrorism deserve despair destroy terror terrific terrible terrorist customer they complain control contact conflict concern con completely comedy cop come tiff coffee coach clown cloud cool could date dismal tho dangerous thought dad thousand threaten cunt cover ticket cross crime create crash crap discourage surprise divide enough episode etc store event stomach every everybody step steal statement exactly state star exhilarate exist expect st spirit spider speak source soul fail faith sort sorry soon some fast entire enjoy dm story term team taste target dont door draw dreadful clear drive drop drug support sunk suck due success dumb early earth style stuck struggle election else em email emotion straight till charlotte tire apparently assume within woman article arm argument app attempt anything anyone anymore worried worry another at average believe bb wall wanna become beautiful war weak ball avoid weekend whatever where white awe will worth animosity animate address yes aggravate yesterday afternoon afraid affliction add animal actually action young account your absolute agree ahs air alarm album all almost yeah yall ya amaze america american wrath an anger would vote voice together try truly catch trump case care card turn centre cannot tv cancel tweet campaign cam celtic challenge bet treat top chirp towards track train trash check chance charlotteprotest snapchat charge character channel true call buy but blind boil usa blue bloody blood block blame business black use useless very bill biggest update bore box unhappy unga breezy bright bring broadcast ugh brother brown btw bully burn burst typical social fire snap lively longer ppl president price lmao pride live lord probably problem promise lip line protest power positive provoke police many please plus pm point madden policy poor ly lunch luck lt low political light pun mary kid race king racism kind kill kik racist rabid key radio just jump joyous rape kitchen lack levity leadership level let punch punny least pure lead lady push put queue quite late quote mark match smoke or nice opportunity next news optimist network negative opinion order near nd nation others otherwise nicole open muslim only offer offense on obviously obama nut nothing no note not north normal nope non must musically mate place minute pessimist peter milk michelle pine play mistake meet medium mean me maybe player peace pay outrage most murder mufc pak movie move pakistan parent paul mope park month money part party joke join rather series gloom send sense sentence give girl george goal serious garbage game seriously fury furious self god set scare sam guy gun gt grudge save group season scary scene grieve great scorn scrum service fuming rd sink floor flight fix single finish finger sky follow final film smell fight smh felt folk food fume free shit shock shoot fuck frown shut freak foot forward side sign force for football safe rush run report incense in important imagine replace ill idiot indian idea id resent human resentment respect india info hang iphone jesus jazz issue islam reason irritate involve infuriate intimidate internet insult instead record relentless house respond horror he hell hearty room heard hear rooney haunt horrific hath round hat ruin rule happen rojo hey heyday hi rock road hilarity hillary rn rip hold ring revolt honestly return horrible result zero",
    "user people like say terrorism fuck really amp black outrage terror trump way make need want take get think try offend use war you pakistan anger the let police bully look word fury it if hell go not kill die woman give fear nothing insult problem serious right would no talk he this they and awful irritate someone pay ask white joke man world we country but shit sad fight attack state stop well something yes protest seriously concern all support fume boil shoot blood racism don revenge never india why vote hillary speak destroy yeah can hope terrorist threaten money sure human king keep afraid thing reason what mean know men gun better drive sort phone provoke cop that racist america tweet towards away point wait pretty decide date rabid break could resent weak hand even might soul rule miss call bitch come future class truth ppl some shot party levity care suppose maybe wanna on rag child cheer etc help force crime bright charlotte ugh without fuming these power be hold fact instead line sober president train side scene muslim centre sense islam mistake usa join unga peace despair whole day only race election must obviously respect agree nation jump mine resentment fake in brother bad chirp indian charlotteprotest many opinion lead every officer or pak save affliction gloom rush kinda conflict non order obama situation promise stuff shut bloody address exactly honestly deserve info for murder blind ignore policy animosity revolt stupid target offense offer spider spirit parent em lunch outside idiot id statement big rape dangerous american biggest leave guy average otherwise typical form scorn fiery wrath hath treat add sam smoke imagine argument carry express sentence enjoy term happen self seem due where level hot poor horrific moment freak divide shock bore whatever just ruin back quote shy pull growl exist pride put matter online cut intimidate depress very dont case nightmare dude brown nigga city hey front plan mother light report apparently check see cheerfully business guess act thought fashion emotion buy kind pick laugh accept secret experience lt finger weekend fire football waste wtf gym im broken job kid rage rapper grudge good cheerful time feeling gloomy hate felt gleeful forward glee already afternoon fill hat glad age film girl gt feel ago goal head fall grim grieve family absolute grind great gotta group fan absolutely grow far have haunt gonna god fast favorite adult aggravate actually george folk first fuckin fix album flight floor frown focus half follow forget food foot fright friend friday hang free account forever fucking hair alarm haha also ahs gbbo final garbage finally find hard fine game furious funny fun action finish air full alone happy almost bb faith cheery character channel change chance arm challenge celtic cause catch cat card car cant cannot candice cancel campaign charge chuckle around clear anyone cool control contact anything anyway con completely complain comedy college coffee coach clown cloud app close cam article fail bitter birthday bill awe awesome awkward bet best believe behind begin bee bed ball become beautiful beat bc bit blame bus block burst burn as assume btw at broadcast bring breezy attempt boy box book avoid body blue blow cover crap crash end email else elate either eat easy earth early dumb dull dry drunk an drug drop bar drink empty enough create entire facebook face eye extra expect exhilarate exhaust excite exact always everything everyone everybody amaze ever event episode dream angry animal dreadful anxiety debate death dear deal dead heard dark damn dad cuz cute customer cunt cry anymore cross answer deep another dm dread draw door animate dollar dog do dismal definitely discourage different difference annoy depression depressing delight hear zero heart sister so snapchat snap smile smh smell sleep sky sit sink somebody single sing since simple sign sick show short she social sometimes shake start straight story store stomach sting still step steal stay star son stand st spend sparkle source sound sorry soon song shall set struggle relentless reverse return result restless rest respond replace remind remember relationship rip rejoice red record realize reality real ready read reach ring rn service saw series send second season scrum school scary scare scar saturday road safe sadness sadly run round rooney room rojo rock stress stuck hearty wake when week website weary wear water watch wall walk voice will video val useless us ur update unhappy understand two who win tv write your young yo yet yesterday year yall ya wrong wow wish worth worst worse worry worried work wonder within with twitter turn style sweet th texan test terrific terrible tell tear team taste swear thanks surprise super sunk summer sulk suggest suffer suck success thank then truly tired true trash track touch top tonight tomorrow together today tire there tip till tiff ticket throw three thousand though tho rd rather rap little yr longer long lonely lol lmao living lively live literally lose listen lip life lie less least learn leadership lay lord loser later mary mess memory member meet medium me may mate match mark lot manchester major madden mad machine ly luck low love laughter late radio horrid important ill idea hurt huff how house hour horror horrible infuriate home holiday hit history hilarity hilarious high hi heyday incense inside last joy lady lack kitchen kiss kik kick key joyous joyful john instagram jesus jazz issue irritation irate iphone involve interview internet mhchat michelle milk paul piss pine picture pic photo peter pessimist person perfect past place part park panic pain others option optimist optimism opportunity pizza play min probably quite queue question push pure punny punch pun process price player pout post positive pop political pm plus please playful open one omg most name my musically music mufc much movie move mourn morning old mope mood month monday momma mom mirth minute mind nd near negative nervous okay ok oh often office nut now note north normal nope night nicole nice ni next news new network able",
    "user get go amp the back like would blue know time love new it start horror look work come turn play girl every fuck good today try year always great this next could hilarious furious say old sorry watch please alarm angry sad happen wish run just think thank kid night shit leave oh burn guy so house move movie player follow us wrong day expect you free late last lol see show one there tonight much many man that tv dreadful nightmare wanna home if rage do month buy call haunt awe bully fire first meet as idea game hilarity dark terrible ticket smile frown ok be wait bring another panic become breezy flight sit away horrible tweet face lip listen animate open absolutely music lmao god match sing relentless remember optimism room nd story serious fume delight together end discourage hear dream week pm put no news th video place grim gbbo top keep boy still gonna felt rejoice hour ball afternoon soon rather episode madden door pessimist crap social film half take safe food write funny ya anything mind kind book dont box foot mean create season scary hey rock step two morning friend early finish pure infuriate long lot depress interview attempt wonder clown fast stop style app your milk voice absolute negative beautiful lady rest nervous make maybe may mate aggravate hope wow rooney air head post lord find block coffee st trash dread deep debate fan swear track dad taste thanks crash forward friday set shock clear shake holiday in short cloud can not mark longer heyday believe stuck star an joyous radio cam dumb series hi steal easy age happy we hell mufc till tire tiff tomorrow minute terrific catch account tho channel animal nice store punch opportunity thousand nut poor campaign celtic loser political album normal surprise ahs park comedy mary cool pine cunt close check floor beat btw round where ill sky fix fill rojo website rd smell jazz email wall folk scrum have twitter scare far part bus yall weekend finger hit kill source win luck rip contact football brown depressing order incense exhilarate punny city pun smh parent saw optimist yeah single eat memory song little enough business imagine cut sometimes finally instead seriously fright anymore who john within let life report yesterday weary super then avoid apparently but fail divide medium due hang all definitely able others trump rap useless pride shoot sure kiss amaze agree promise he me tear anyway date sweet suppose deal dear talk death dangerous decide damn dead often support target dog suck dm suffer dismal suggest different difference die destroy despair deserve depression sulk summer sunk customer cuz cute completely complain texan these college they coach thing though thought threaten class three chuckle chirp con concern conflict term team don cry cross crime tell terror test cover country terrorism cop terrorist control dollar draw success faith fact facebook sound eye extra express experience sparkle exist speak exhaust excite exactly exact everything soul fake everybody fall final some somebody fight fiery someone feeling feel fear favorite something fashion son sort family everyone spend stupid stay sting stomach dull straight dude dry drunk drug drop drive drink stress struggle stuff cheery earth statement ever either event even etc spider entire spirit enjoy stand empty emotion state em else election elate child charlotteprotest cheerfully animosity assume ask what article around arm argument whatever when white anyone whole anxiety answer why at attack well bad want war bc bb bar waste water average awkward awful awesome way weak wear annoy will cheerful with wrath wtf yes afraid affliction adult address add actually action act yet accept yo young ago worth worst america without anger and woman word american world worse also already alone almost worried worry walk wake bed bee card car cant cannot candice cancel towards train treat true truly truth typical burst ugh care carry case tip cheer throw snapchat charlotte charge character change cat chance challenge centre tired cause touch understand unga unhappy biggest black bitter bitch bit birthday bill big blind better bet best vote behind begin blame very brother usa broken broadcast update bright ur break use blood bore boil body val blow bloody sober sleep fine lt positive pout power lose ppl yr president pretty lonely price living lively live probably literally low pop snap lunch picture matter piss pizza plan playful plus manchester point police major policy mad machine ly problem process line light lack quite kitchen king kinda quote rabid kik race kick key racism racist jump rag queue question push pull protest lie levity level provoke less least later learn leadership lead lay laughter laugh pick pic photo must nigga nicole option ni or otherwise never network outrage outside need near pain nation name opinion only online okay office offer offense offend obviously obama now non nothing note omg north on nope my muslim phone musically mistake miss mirth people mine perfect min person might michelle mhchat mess men peter member mom moment momma mourn pak murder pakistan party past paul mother monday most pay mope mood peace money joyful joy joke seem save scar scene goal school gloomy gloom gleeful glee glad give scorn second george secret gotta saturday grieve gun hand sadly hair haha gym sadness guess sam gt grudge growl grow group grind garbage future rule fury forever force for shy sick side sign simple focus since sink sister situation officer smoke forget form shut fucking self send fun fuming sense full fuckin freak sentence service front shall she shot rush ruin join human ready info indian india real reality important realize im really ignore reason idiot record id inside instagram read irritation job jesus rape rapper issue islam irritate insult irate iphone involve intimidate reach internet hurt huff hard how return revenge reverse help revolt hearty heart heard right ring rn road hath hate hat high result restless replace red relationship hot remind horrific horrid resent hillary honestly resentment respect hold respond history zero",
    "user get live watch think worry one know want amaze fear let never horrible even ever anger team life give shock terrible snap service broadcast musically time game ly see the angry unhappy still awful you play what well don customer yet lively someone worst tell bad use actually would anyone my bitter gt guy back afraid rage hold issue part people fuck us god sadly depression quote call best sink bb discourage drop when send point else that take but just least anything put snapchat burst minute deal at another sadness how come win person enough true chance stop probably grudge season lie wrath he lol already they may nothing read news despair straight key feel paul kik please cover almost believe also medium mean struggle dm suck okay sunk horrific be pic last everything within cross road heard she change hear bet success offense side record no care month nicole fuming wow faith level cause involve history maybe nope sign show peter contact final result plus lt fume control zero american texan store truly michelle hand con complain hang happen entire coach lack everybody assume forever year hearty blame action intimidate dismal network group test big will replace respond drug goal return bill low draw price optimism house challenge arm smh card around help experience internet near anymore charge now must cancel email stomach push kitchen thought honestly pine haha worried earth positive garbage important george note cannot rn north completely fail queue mope since exhilarate bore iphone character jesus update have event article thanks hat your name young most quite leadership worth ring death behind biggest st grieve moment avoid scare irate optimist blow perfect useless me business literally heart saturday better top rip pay high emotion damn freak tire voice others very focus reach kind self talk sweet ruin yesterday case light ago dear whatever shy incense single exist less rojo fine fuckin joy dull address leave forget guess everyone horrid lmao hey force for form absolutely hell accept account forward head act annoy haunt hath football hillary heyday foot finally hour find hot horror finger answer hope finish home fire holiday first absolute hit fix flight floor folk follow friday hilarity hilarious food hi free grim friend gloomy affliction girl alone glad all afternoon growl glee age gleeful gloom go gbbo aggravate album gonna grow good alarm gotta agree air ahs great always gun hate full fright hard add happy grind front frown animosity fucking half hair animate america animal and an amp gym fun funny furious fury future adult btw excite anxiety crap country could black cop cool conflict concern comedy blind block college coffee blood bloody clown cloud close bitch crash depress create deep decide debate dead day date dark dangerous dad cuz cute cut cunt cry birthday bit crime clear class city chuckle bright catch cat carry bring car cant candice can campaign cam broken buy brother bus brown burn breezy celtic centre boil chirp blue child body cheery cheerfully cheerful cheer break check book charlotteprotest charlotte box channel boy definitely depressing film as eye extra ask express expect exhaust bully exactly exact every attack etc attempt episode average away enjoy face facebook deserve fact fill fight fiery felt feeling anyway app apparently argument favorite fast fashion far fan family fall fake awe end empty awesome dream dreadful dread door dont dollar dog do become bed bee divide begin different difference die destroy beautiful drink drive bar em awkward election elate either eat ball easy beat bc early dumb due dude dry drunk delight look huff smile song son sometimes something somebody some social sober so smoke smell shoot sleep sky situation sit sister sing simple sick shut shot soon sorry sort soul stupid stuff stuck stress story sting step steal stay statement state start star stand spirit spider spend speak sparkle source sound short shit suffer respect round rooney room rock right revolt reverse revenge restless rest resentment shall resent report remind remember relentless relationship rejoice red reason really rule run rush sad shake set seriously serious series sentence sense seem secret second scrum scorn school scene scary scar say saw save sam safe style suggest reality walk weary wear weak we way water waste war wanna wall wake twitter wait vote video val usa ur unga understand ugh typical website week weekend where yo yes yeah yall ya wtf wrong write worse world work word wonder woman without with wish why whole who white two tweet sulk terrific this thing these there then thank th terrorist terrorism terror term tv tear taste target swear surprise sure suppose support super summer tho though thousand threaten turn try truth trump treat trash train track towards touch tonight tomorrow together today tired tip till tiff ticket throw three realize real human luck mark many manchester man make major madden mad machine lunch love lip lot loser lose lord yr longer long lonely living little mary match mate matter mother morning mood money monday momma mom mistake miss mirth mine mind min milk might mhchat mess men memory member meet listen line move indian islam irritation irritate interview insult instead instagram inside infuriate info india like in imagine im ill ignore if idiot idea id hurt it jazz job john levity learn lead lay laughter laugh later late lady kiss king kinda kill kid kick keep jump joyous joyful joke join mourn movie ready playful ppl power pout post pop poor political policy police pm player party plan place pizza piss picture pick photo phone pessimist peace president pretty pride problem rd rather rapper rape rap rag radio racist racism race rabid question pure punny punch pun pull provoke protest promise process past park much ni obviously obama nut not normal non nightmare night nigga nice next parent new nervous negative need nd nation muslim music murder mufc offend offer office officer panic pakistan pak pain outside outrage otherwise order or option opportunity opinion open only online on omg old ok oh often able"
]

unseeded_model_5 = [
    "user get like make go feel lose day one amp love see thing life anxiety im depression sad even people find need it work much really sadness happy good the today my little right hate look still time say do cry tell think start when take hard want smile stay shake week way night heart someone depress sleep be know something face glee ur year real family literally bit first nightmare you birthday bc pout everyone miss talk tomorrow job we friend cheer twitter new mourn how home lol angry since also either never always though well last nervous leave why end fear so change break post man sting die tear oh and around gloomy school worse worst bad help big sometimes phone long fall walk ask watch question eye thanks ever wait panic person mind yo she sound pretty gbbo lie laughter sober head forget revenge shy understand two wake car best playful fun rag laugh joyful mad seem full world tonight lot weary if half annoy hour dull this attack show alone fact us sparkle body sick hit excite hair hilarious horrid past dream gotta realize without hurt restless cant inside rage wtf gleeful cause whole stand music spend don mhchat piss outside stop age omg eat begin fucking now grim grow scar ready morning bed old keep cheery funny gonna with feeling bitter test lonely already child saw wish learn tip pizza relationship water remember name act ni away wear boy might front fuckin fan adult often depressing min later dog major hope pop offend val cat huff bar process yr mirth red broken mom picture photo dry guess everything that suppose cute second answer close extra damn different son glad finally high somebody bee machine kick kinda ago moment dear fiery death listen sister book pick member living dark who as three college stress summer suggest dude awesome tired awful awkward mood growl elate back drunk reality drink pain favorite empty accept dollar office sulk sure far manchester suffer matter option stuff difference exact exhaust plan mess stupid facebook holiday then instagram cuz kiss throw rapper truth irritation rejoice candice remind write sadly rest chuckle touch shall reverse can lay simple less dead pull read online focus class gym super sit grind song anyway cheerful waste okay monday momma cheerfully sweet future hand history city definitely fine forever experience john bus fill fright joy able win word rap wow reach secret website may hot form memory swear have fashion nigga blow short dread mother what bitch fake perfect mine shot friday beat cut carry men wonder haha absolutely interview situation irate saturday express pic loser easy yet wrong sing country ignore optimism video behind better ok thank deal there these debate officer decide deep th delight texan terrorism deserve despair destroy terror terrific terrible terrorist customer they complain control contact conflict concern con completely comedy cop come tiff coffee coach clown cloud cool could date dismal tho dangerous thought dad thousand threaten cunt cover ticket cross crime create crash crap discourage surprise divide enough episode etc store event stomach every everybody step steal statement exactly state star exhilarate exist expect st spirit spider speak source soul fail faith sort sorry soon some fast entire enjoy dm story term team taste target dont door draw dreadful clear drive drop drug support sunk suck due success dumb early earth style stuck struggle election else em email emotion straight till charlotte tire apparently assume within woman article arm argument app attempt anything anyone anymore worried worry another at average believe bb wall wanna become beautiful war weak ball avoid weekend whatever where white awe will worth animosity animate address yes aggravate yesterday afternoon afraid affliction add animal actually action young account your absolute agree ahs air alarm album all almost yeah yall ya amaze america american wrath an anger would vote voice together try truly catch trump case care card turn centre cannot tv cancel tweet campaign cam celtic challenge bet treat top chirp towards track train trash check chance charlotteprotest snapchat charge character channel true call buy but blind boil usa blue bloody blood block blame business black use useless very bill biggest update bore box unhappy unga breezy bright bring broadcast ugh brother brown btw bully burn burst typical social fire snap lively longer ppl president price lmao pride live lord probably problem promise lip line protest power positive provoke police many please plus pm point madden policy poor ly lunch luck lt low political light pun mary kid race king racism kind kill kik racist rabid key radio just jump joyous rape kitchen lack levity leadership level let punch punny least pure lead lady push put queue quite late quote mark match smoke or nice opportunity next news optimist network negative opinion order near nd nation others otherwise nicole open muslim only offer offense on obviously obama nut nothing no note not north normal nope non must musically mate place minute pessimist peter milk michelle pine play mistake meet medium mean me maybe player peace pay outrage most murder mufc pak movie move pakistan parent paul mope park month money part party joke join rather series gloom send sense sentence give girl george goal serious garbage game seriously fury furious self god set scare sam guy gun gt grudge save group season scary scene grieve great scorn scrum service fuming rd sink floor flight fix single finish finger sky follow final film smell fight smh felt folk food fume free shit shock shoot fuck frown shut freak foot forward side sign force for football safe rush run report incense in important imagine replace ill idiot indian idea id resent human resentment respect india info hang iphone jesus jazz issue islam reason irritate involve infuriate intimidate internet insult instead record relentless house respond horror he hell hearty room heard hear rooney haunt horrific hath round hat ruin rule happen rojo hey heyday hi rock road hilarity hillary rn rip hold ring revolt honestly return horrible result zero",
    "user people like say terrorism fuck really amp black outrage terror trump way make need want take get think try offend use war you pakistan anger the let police bully look word fury it if hell go not kill die woman give fear nothing insult problem serious right would no talk he this they and awful irritate someone pay ask white joke man world we country but shit sad fight attack state stop well something yes protest seriously concern all support fume boil shoot blood racism don revenge never india why vote hillary speak destroy yeah can hope terrorist threaten money sure human king keep afraid thing reason what mean know men gun better drive sort phone provoke cop that racist america tweet towards away point wait pretty decide date rabid break could resent weak hand even might soul rule miss call bitch come future class truth ppl some shot party levity care suppose maybe wanna on rag child cheer etc help force crime bright charlotte ugh without fuming these power be hold fact instead line sober president train side scene muslim centre sense islam mistake usa join unga peace despair whole day only race election must obviously respect agree nation jump mine resentment fake in brother bad chirp indian charlotteprotest many opinion lead every officer or pak save affliction gloom rush kinda conflict non order obama situation promise stuff shut bloody address exactly honestly deserve info for murder blind ignore policy animosity revolt stupid target offense offer spider spirit parent em lunch outside idiot id statement big rape dangerous american biggest leave guy average otherwise typical form scorn fiery wrath hath treat add sam smoke imagine argument carry express sentence enjoy term happen self seem due where level hot poor horrific moment freak divide shock bore whatever just ruin back quote shy pull growl exist pride put matter online cut intimidate depress very dont case nightmare dude brown nigga city hey front plan mother light report apparently check see cheerfully business guess act thought fashion emotion buy kind pick laugh accept secret experience lt finger weekend fire football waste wtf gym im broken job kid rage rapper grudge good cheerful time feeling gloomy hate felt gleeful forward glee already afternoon fill hat glad age film girl gt feel ago goal head fall grim grieve family absolute grind great gotta group fan absolutely grow far have haunt gonna god fast favorite adult aggravate actually george folk first fuckin fix album flight floor frown focus half follow forget food foot fright friend friday hang free account forever fucking hair alarm haha also ahs gbbo final garbage finally find hard fine game furious funny fun action finish air full alone happy almost bb faith cheery character channel change chance arm challenge celtic cause catch cat card car cant cannot candice cancel campaign charge chuckle around clear anyone cool control contact anything anyway con completely complain comedy college coffee coach clown cloud app close cam article fail bitter birthday bill awe awesome awkward bet best believe behind begin bee bed ball become beautiful beat bc bit blame bus block burst burn as assume btw at broadcast bring breezy attempt boy box book avoid body blue blow cover crap crash end email else elate either eat easy earth early dumb dull dry drunk an drug drop bar drink empty enough create entire facebook face eye extra expect exhilarate exhaust excite exact always everything everyone everybody amaze ever event episode dream angry animal dreadful anxiety debate death dear deal dead heard dark damn dad cuz cute customer cunt cry anymore cross answer deep another dm dread draw door animate dollar dog do dismal definitely discourage different difference annoy depression depressing delight hear zero heart sister so snapchat snap smile smh smell sleep sky sit sink somebody single sing since simple sign sick show short she social sometimes shake start straight story store stomach sting still step steal stay star son stand st spend sparkle source sound sorry soon song shall set struggle relentless reverse return result restless rest respond replace remind remember relationship rip rejoice red record realize reality real ready read reach ring rn service saw series send second season scrum school scary scare scar saturday road safe sadness sadly run round rooney room rojo rock stress stuck hearty wake when week website weary wear water watch wall walk voice will video val useless us ur update unhappy understand two who win tv write your young yo yet yesterday year yall ya wrong wow wish worth worst worse worry worried work wonder within with twitter turn style sweet th texan test terrific terrible tell tear team taste swear thanks surprise super sunk summer sulk suggest suffer suck success thank then truly tired true trash track touch top tonight tomorrow together today tire there tip till tiff ticket throw three thousand though tho rd rather rap little yr longer long lonely lol lmao living lively live literally lose listen lip life lie less least learn leadership lay lord loser later mary mess memory member meet medium me may mate match mark lot manchester major madden mad machine ly luck low love laughter late radio horrid important ill idea hurt huff how house hour horror horrible infuriate home holiday hit history hilarity hilarious high hi heyday incense inside last joy lady lack kitchen kiss kik kick key joyous joyful john instagram jesus jazz issue irritation irate iphone involve interview internet mhchat michelle milk paul piss pine picture pic photo peter pessimist person perfect past place part park panic pain others option optimist optimism opportunity pizza play min probably quite queue question push pure punny punch pun process price player pout post positive pop political pm plus please playful open one omg most name my musically music mufc much movie move mourn morning old mope mood month monday momma mom mirth minute mind nd near negative nervous okay ok oh often office nut now note north normal nope night nicole nice ni next news new network able",
    "user get go amp the back like would blue know time love new it start horror look work come turn play girl every fuck good today try year always great this next could hilarious furious say old sorry watch please alarm angry sad happen wish run just think thank kid night shit leave oh burn guy so house move movie player follow us wrong day expect you free late last lol see show one there tonight much many man that tv dreadful nightmare wanna home if rage do month buy call haunt awe bully fire first meet as idea game hilarity dark terrible ticket smile frown ok be wait bring another panic become breezy flight sit away horrible tweet face lip listen animate open absolutely music lmao god match sing relentless remember optimism room nd story serious fume delight together end discourage hear dream week pm put no news th video place grim gbbo top keep boy still gonna felt rejoice hour ball afternoon soon rather episode madden door pessimist crap social film half take safe food write funny ya anything mind kind book dont box foot mean create season scary hey rock step two morning friend early finish pure infuriate long lot depress interview attempt wonder clown fast stop style app your milk voice absolute negative beautiful lady rest nervous make maybe may mate aggravate hope wow rooney air head post lord find block coffee st trash dread deep debate fan swear track dad taste thanks crash forward friday set shock clear shake holiday in short cloud can not mark longer heyday believe stuck star an joyous radio cam dumb series hi steal easy age happy we hell mufc till tire tiff tomorrow minute terrific catch account tho channel animal nice store punch opportunity thousand nut poor campaign celtic loser political album normal surprise ahs park comedy mary cool pine cunt close check floor beat btw round where ill sky fix fill rojo website rd smell jazz email wall folk scrum have twitter scare far part bus yall weekend finger hit kill source win luck rip contact football brown depressing order incense exhilarate punny city pun smh parent saw optimist yeah single eat memory song little enough business imagine cut sometimes finally instead seriously fright anymore who john within let life report yesterday weary super then avoid apparently but fail divide medium due hang all definitely able others trump rap useless pride shoot sure kiss amaze agree promise he me tear anyway date sweet suppose deal dear talk death dangerous decide damn dead often support target dog suck dm suffer dismal suggest different difference die destroy despair deserve depression sulk summer sunk customer cuz cute completely complain texan these college they coach thing though thought threaten class three chuckle chirp con concern conflict term team don cry cross crime tell terror test cover country terrorism cop terrorist control dollar draw success faith fact facebook sound eye extra express experience sparkle exist speak exhaust excite exactly exact everything soul fake everybody fall final some somebody fight fiery someone feeling feel fear favorite something fashion son sort family everyone spend stupid stay sting stomach dull straight dude dry drunk drug drop drive drink stress struggle stuff cheery earth statement ever either event even etc spider entire spirit enjoy stand empty emotion state em else election elate child charlotteprotest cheerfully animosity assume ask what article around arm argument whatever when white anyone whole anxiety answer why at attack well bad want war bc bb bar waste water average awkward awful awesome way weak wear annoy will cheerful with wrath wtf yes afraid affliction adult address add actually action act yet accept yo young ago worth worst america without anger and woman word american world worse also already alone almost worried worry walk wake bed bee card car cant cannot candice cancel towards train treat true truly truth typical burst ugh care carry case tip cheer throw snapchat charlotte charge character change cat chance challenge centre tired cause touch understand unga unhappy biggest black bitter bitch bit birthday bill big blind better bet best vote behind begin blame very brother usa broken broadcast update bright ur break use blood bore boil body val blow bloody sober sleep fine lt positive pout power lose ppl yr president pretty lonely price living lively live probably literally low pop snap lunch picture matter piss pizza plan playful plus manchester point police major policy mad machine ly problem process line light lack quite kitchen king kinda quote rabid kik race kick key racism racist jump rag queue question push pull protest lie levity level provoke less least later learn leadership lead lay laughter laugh pick pic photo must nigga nicole option ni or otherwise never network outrage outside need near pain nation name opinion only online okay office offer offense offend obviously obama now non nothing note omg north on nope my muslim phone musically mistake miss mirth people mine perfect min person might michelle mhchat mess men peter member mom moment momma mourn pak murder pakistan party past paul mother monday most pay mope mood peace money joyful joy joke seem save scar scene goal school gloomy gloom gleeful glee glad give scorn second george secret gotta saturday grieve gun hand sadly hair haha gym sadness guess sam gt grudge growl grow group grind garbage future rule fury forever force for shy sick side sign simple focus since sink sister situation officer smoke forget form shut fucking self send fun fuming sense full fuckin freak sentence service front shall she shot rush ruin join human ready info indian india real reality important realize im really ignore reason idiot record id inside instagram read irritation job jesus rape rapper issue islam irritate insult irate iphone involve intimidate reach internet hurt huff hard how return revenge reverse help revolt hearty heart heard right ring rn road hath hate hat high result restless replace red relationship hot remind horrific horrid resent hillary honestly resentment respect hold respond history zero",
    "user get live watch think worry one know want amaze fear let never horrible even ever anger team life give shock terrible snap service broadcast musically time game ly see the angry unhappy still awful you play what well don customer yet lively someone worst tell bad use actually would anyone my bitter gt guy back afraid rage hold issue part people fuck us god sadly depression quote call best sink bb discourage drop when send point else that take but just least anything put snapchat burst minute deal at another sadness how come win person enough true chance stop probably grudge season lie wrath he lol already they may nothing read news despair straight key feel paul kik please cover almost believe also medium mean struggle dm suck okay sunk horrific be pic last everything within cross road heard she change hear bet success offense side record no care month nicole fuming wow faith level cause involve history maybe nope sign show peter contact final result plus lt fume control zero american texan store truly michelle hand con complain hang happen entire coach lack everybody assume forever year hearty blame action intimidate dismal network group test big will replace respond drug goal return bill low draw price optimism house challenge arm smh card around help experience internet near anymore charge now must cancel email stomach push kitchen thought honestly pine haha worried earth positive garbage important george note cannot rn north completely fail queue mope since exhilarate bore iphone character jesus update have event article thanks hat your name young most quite leadership worth ring death behind biggest st grieve moment avoid scare irate optimist blow perfect useless me business literally heart saturday better top rip pay high emotion damn freak tire voice others very focus reach kind self talk sweet ruin yesterday case light ago dear whatever shy incense single exist less rojo fine fuckin joy dull address leave forget guess everyone horrid lmao hey force for form absolutely hell accept account forward head act annoy haunt hath football hillary heyday foot finally hour find hot horror finger answer hope finish home fire holiday first absolute hit fix flight floor folk follow friday hilarity hilarious food hi free grim friend gloomy affliction girl alone glad all afternoon growl glee age gleeful gloom go gbbo aggravate album gonna grow good alarm gotta agree air ahs great always gun hate full fright hard add happy grind front frown animosity fucking half hair animate america animal and an amp gym fun funny furious fury future adult btw excite anxiety crap country could black cop cool conflict concern comedy blind block college coffee blood bloody clown cloud close bitch crash depress create deep decide debate dead day date dark dangerous dad cuz cute cut cunt cry birthday bit crime clear class city chuckle bright catch cat carry bring car cant candice can campaign cam broken buy brother bus brown burn breezy celtic centre boil chirp blue child body cheery cheerfully cheerful cheer break check book charlotteprotest charlotte box channel boy definitely depressing film as eye extra ask express expect exhaust bully exactly exact every attack etc attempt episode average away enjoy face facebook deserve fact fill fight fiery felt feeling anyway app apparently argument favorite fast fashion far fan family fall fake awe end empty awesome dream dreadful dread door dont dollar dog do become bed bee divide begin different difference die destroy beautiful drink drive bar em awkward election elate either eat ball easy beat bc early dumb due dude dry drunk delight look huff smile song son sometimes something somebody some social sober so smoke smell shoot sleep sky situation sit sister sing simple sick shut shot soon sorry sort soul stupid stuff stuck stress story sting step steal stay statement state start star stand spirit spider spend speak sparkle source sound short shit suffer respect round rooney room rock right revolt reverse revenge restless rest resentment shall resent report remind remember relentless relationship rejoice red reason really rule run rush sad shake set seriously serious series sentence sense seem secret second scrum scorn school scene scary scar say saw save sam safe style suggest reality walk weary wear weak we way water waste war wanna wall wake twitter wait vote video val usa ur unga understand ugh typical website week weekend where yo yes yeah yall ya wtf wrong write worse world work word wonder woman without with wish why whole who white two tweet sulk terrific this thing these there then thank th terrorist terrorism terror term tv tear taste target swear surprise sure suppose support super summer tho though thousand threaten turn try truth trump treat trash train track towards touch tonight tomorrow together today tired tip till tiff ticket throw three realize real human luck mark many manchester man make major madden mad machine lunch love lip lot loser lose lord yr longer long lonely living little mary match mate matter mother morning mood money monday momma mom mistake miss mirth mine mind min milk might mhchat mess men memory member meet listen line move indian islam irritation irritate interview insult instead instagram inside infuriate info india like in imagine im ill ignore if idiot idea id hurt it jazz job john levity learn lead lay laughter laugh later late lady kiss king kinda kill kid kick keep jump joyous joyful joke join mourn movie ready playful ppl power pout post pop poor political policy police pm player party plan place pizza piss picture pick photo phone pessimist peace president pretty pride problem rd rather rapper rape rap rag radio racist racism race rabid question pure punny punch pun pull provoke protest promise process past park much ni obviously obama nut not normal non nightmare night nigga nice next parent new nervous negative need nd nation muslim music murder mufc offend offer office officer panic pakistan pak pain outside outrage otherwise order or option opportunity opinion open only online on omg old ok oh often able"
]

In [21]:
unseeded_model_1 = [topic.split() for topic in unseeded_model_1]
unseeded_model_2 = [topic.split() for topic in unseeded_model_2]
unseeded_model_3 = [topic.split() for topic in unseeded_model_3]
unseeded_model_4 = [topic.split() for topic in unseeded_model_4]
unseeded_model_5 = [topic.split() for topic in unseeded_model_5]

In [22]:
# all of these are list of list and now contained in another list

unseeded_model_topics_list = [unseeded_model_1, unseeded_model_2, unseeded_model_3, unseeded_model_4, unseeded_model_5]

In [23]:
for i, unseeded_model_topics in enumerate(unseeded_model_topics_list):
    print(f"[Info:] Working with unseeded model {i+1}")
    print("----------------------------\n")
    overall_topic = ""
    overall_max = -1.0
    for i, topic in enumerate(unseeded_model_topics):
        max = -1.0
        topic_name = ""
        for key, value in seed_word.items():
            common_words = set(topic) & set(value)
            common_words_count = len(common_words)
            percentage = (common_words_count / len(value)) * 100
            print(f"For Topic {i}, Percentage of words in emotion: {key.upper()} = {percentage: .2f}%")
            print(f"Common words are: {common_words}")
            if max < percentage:
                max = percentage
                topic_name = key
        if overall_max < max:
            overall_max = max
            overall_topic = topic_name
        print("\n=====================\n")
    print("|||||||||||||||||||||||||||||||||||||||||||||||||||")
    print(f"[NOTE:] Dominant topic in unseeded model {i+1} is {overall_topic.upper()} with pribability {overall_max:.2f}%")
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n")

[Info:] Working with unseeded model 1
----------------------------

For Topic 0, Percentage of words in emotion: ANGER =  17.07%
Common words are: {'angry', 'mad', 'irate', 'bitter', 'outrage', 'cross', 'furious'}
For Topic 0, Percentage of words in emotion: SADNESS =  11.11%
Common words are: {'unhappy', 'sad', 'blue', 'dismal', 'gloomy'}
For Topic 0, Percentage of words in emotion: JOY =  13.33%
Common words are: {'positive', 'joyful', 'gleeful', 'happy', 'cheery', 'cheerful'}
For Topic 0, Percentage of words in emotion: OPTIMISM =  8.89%
Common words are: {'joyful', 'cheerful', 'bright', 'positive'}


For Topic 1, Percentage of words in emotion: ANGER =  17.07%
Common words are: {'angry', 'mad', 'irate', 'bitter', 'outrage', 'cross', 'furious'}
For Topic 1, Percentage of words in emotion: SADNESS =  11.11%
Common words are: {'unhappy', 'sad', 'blue', 'dismal', 'gloomy'}
For Topic 1, Percentage of words in emotion: JOY =  13.33%
Common words are: {'positive', 'joyful', 'gleeful', 'ha

As all the models from task 3 are giving same results I am picking the first model.

And then with the topics of that model will calculate the confusion matrix for both seeded and unseeded model

In [33]:
emotion_df["label"].unique()

array(['optimism', 'anger', 'joy', 'sadness'], dtype=object)

In [34]:
true_emotions = emotion_df["label"]
pred_emotions = ["anger" for _ in range(len(true_emotions))]

In [35]:
confusion_matrix(true_emotions, pred_emotions)

array([[1400,    0,    0,    0],
       [ 708,    0,    0,    0],
       [ 294,    0,    0,    0],
       [ 855,    0,    0,    0]])

Same for the model with seeded lda too. So, the predictions are not good enough from the model.
We need more contextual introduction to understand rather pure statistical analysis